# Data work



## DNB Data

In [ ]:
import pandas as pd
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQeysOyp_m6olZ3seqX5Ikd3r0J9htChodIu8JH1H_LvEiIE7g8AvVi8iAs_BYQJw/pub?output=csv'
dfin = pd.read_csv(url)
dfin = dfin.drop(dfin.index[-2:])
dfin['Period'] = pd.to_datetime(dfin['Period'], format='%Y').dt.year
dfin

,Geography,Country,SPEs,Period,Value
0,Asia,China,Including SPEs,2015,20866.0
1,Asia,China,Excluding SPEs,2015,1843.0
2,Asia,Philippines,Including SPEs,2015,3769.0
3,Asia,Philippines,Excluding SPEs,2015,140.0
4,Asia,"Hong Kong, China",Including SPEs,2015,38124.0
...,...,...,...,...,...
989,Other Europe,Switzerland,Excluding SPEs,2021,73358.0
990,Other Europe,Other countries,Including SPEs,2021,11129.0
991,Other Europe,Other countries,Excluding SPEs,2021,2485.0
992,Total,Total,Including SPEs,2021,3824322.0


In [ ]:
dfin.loc[dfin['SPEs'] == 'Including SPEs', 'Including SPEs'] = dfin['Value']
dfin.loc[dfin['SPEs'] == 'Excluding SPEs', 'Excluding SPEs'] = dfin['Value']

dfin = dfin.drop(['SPEs', 'Value'], axis=1)
dfin = dfin.groupby(['Geography', 'Country', 'Period']).sum().reset_index()
dfin = dfin.rename(columns={'Including SPEs': 'IN_ALL'})
dfin = dfin.rename(columns={'Excluding SPEs': 'IN_NON'})
dfin

,Geography,Country,Period,IN_ALL,IN_NON
0,Asia,China,2015,20866.0,1843.0
1,Asia,China,2016,18404.0,1458.0
2,Asia,China,2017,17764.0,603.0
3,Asia,China,2018,18519.0,923.0
4,Asia,China,2019,17272.0,-802.0
...,...,...,...,...,...
492,Total,Total,2017,4101393.0,1248788.0
493,Total,Total,2018,3998868.0,1273992.0
494,Total,Total,2019,4055583.0,1270839.0
495,Total,Total,2020,3687114.0,2217698.0


In [ ]:
dfin.loc[dfin['Country'] == 'Other countries', 'Country'] = dfin.loc[dfin['Country'] == 'Other countries', 'Geography']
dfin = dfin.drop('Geography', axis=1)
dfin

,Country,Period,IN_ALL,IN_NON
0,China,2015,20866.0,1843.0
1,China,2016,18404.0,1458.0
2,China,2017,17764.0,603.0
3,China,2018,18519.0,923.0
4,China,2019,17272.0,-802.0
...,...,...,...,...
492,Total,2017,4101393.0,1248788.0
493,Total,2018,3998868.0,1273992.0
494,Total,2019,4055583.0,1270839.0
495,Total,2020,3687114.0,2217698.0


In [ ]:
dfin['IN_SPE'] = dfin['IN_ALL'] - dfin['IN_NON']
dfin['IN_SPE_PERC'] = dfin['IN_SPE'] * 100 / (abs(dfin['IN_NON'])+abs(dfin['IN_SPE']))
dfin['IN_SPE_PERC_DELTA'] = dfin.groupby('Country')['IN_SPE_PERC'].diff()
dfin['IN_SPE_PERC_CHANGE'] = ((dfin['IN_SPE'] - dfin.groupby('Country')['IN_SPE'].shift(1)) / abs(dfin.groupby('Country')['IN_SPE'].shift(1))) * 100
dfin['IN_ALL_PERC_CHANGE'] = ((dfin['IN_ALL'] - dfin.groupby('Country')['IN_ALL'].shift(1)) / abs(dfin.groupby('Country')['IN_ALL'].shift(1))) * 100
dfin = dfin.round(2)
dfin

,Country,Period,IN_ALL,IN_NON,IN_SPE,IN_SPE_PERC,IN_SPE_PERC_DELTA,IN_SPE_PERC_CHANGE,IN_ALL_PERC_CHANGE
0,China,2015,20866.0,1843.0,19023.0,91.17,NaN,NaN,NaN
1,China,2016,18404.0,1458.0,16946.0,92.08,0.91,-10.92,-11.80
2,China,2017,17764.0,603.0,17161.0,96.61,4.53,1.27,-3.48
3,China,2018,18519.0,923.0,17596.0,95.02,-1.59,2.53,4.25
4,China,2019,17272.0,-802.0,18074.0,95.75,0.74,2.72,-6.73
...,...,...,...,...,...,...,...,...,...
492,Total,2017,4101393.0,1248788.0,2852605.0,69.55,1.19,5.29,3.48
493,Total,2018,3998868.0,1273992.0,2724876.0,68.14,-1.41,-4.48,-2.50
494,Total,2019,4055583.0,1270839.0,2784744.0,68.66,0.52,2.20,1.42
495,Total,2020,3687114.0,2217698.0,1469416.0,39.85,-28.81,-47.23,-9.09


Percentage SPEs denotes the percentage of SPEs inward FDI into NL as a share of gross (in or out) FDI.
If the percentage is negative, SPEs outward FDI was larger than its inward FDI.

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSW3LQ8ckCFatfaI_LXEtR2rGwMZeNl-XvRmioZ80nByk94hBNIUXEgGh1sDgmYUxCJhazEkJ1Q8eas/pub?output=csv'
dfout = pd.read_csv(url)
dfout = dfout.drop(dfout.index[-2:])
dfout['Period'] = pd.to_datetime(dfout['Period'], format='%Y').dt.year
dfout

,Geography,Country,SPEs,Period,waarde
0,Asia,China,Including SPEs,2015,29904.0
1,Asia,China,Excluding SPEs,2015,20998.0
2,Asia,Philippines,Including SPEs,2015,16164.0
3,Asia,Philippines,Excluding SPEs,2015,1762.0
4,Asia,"Hong Kong, China",Including SPEs,2015,22310.0
...,...,...,...,...,...
989,Other Europe,Switzerland,Excluding SPEs,2021,312190.0
990,Other Europe,Other countries,Including SPEs,2021,21438.0
991,Other Europe,Other countries,Excluding SPEs,2021,12872.0
992,Total,Total,Including SPEs,2021,4810890.0


In [ ]:
dfout.loc[dfout['SPEs'] == 'Including SPEs', 'Including SPEs'] = dfout['waarde']
dfout.loc[dfout['SPEs'] == 'Excluding SPEs', 'Excluding SPEs'] = dfout['waarde']

dfout = dfout.drop(['SPEs', 'waarde'], axis=1)
dfout = dfout.groupby(['Geography', 'Country', 'Period']).sum().reset_index()
dfout = dfout.rename(columns={'Including SPEs': 'OUT_ALL'})
dfout = dfout.rename(columns={'Excluding SPEs': 'OUT_NON'})
dfout

,Geography,Country,Period,OUT_ALL,OUT_NON
0,Asia,China,2015,29904.0,20998.0
1,Asia,China,2016,30629.0,19782.0
2,Asia,China,2017,26904.0,17604.0
3,Asia,China,2018,26671.0,16173.0
4,Asia,China,2019,40691.0,23988.0
...,...,...,...,...,...
492,Total,Total,2017,5068540.0,1990322.0
493,Total,Total,2018,4985424.0,1913632.0
494,Total,Total,2019,5135148.0,2013941.0
495,Total,Total,2020,4731164.0,2911108.0


In [ ]:
dfout.loc[dfout['Country'] == 'Other countries', 'Country'] = dfout.loc[dfout['Country'] == 'Other countries', 'Geography']
dfout = dfout.drop('Geography', axis=1)
dfout

,Country,Period,OUT_ALL,OUT_NON
0,China,2015,29904.0,20998.0
1,China,2016,30629.0,19782.0
2,China,2017,26904.0,17604.0
3,China,2018,26671.0,16173.0
4,China,2019,40691.0,23988.0
...,...,...,...,...
492,Total,2017,5068540.0,1990322.0
493,Total,2018,4985424.0,1913632.0
494,Total,2019,5135148.0,2013941.0
495,Total,2020,4731164.0,2911108.0


In [ ]:
import numpy as np

In [ ]:
dfout['OUT_SPE'] = dfout['OUT_ALL'] - dfout['OUT_NON']
dfout['OUT_SPE_PERC'] = dfout['OUT_SPE'] * 100 / (abs(dfout['OUT_NON'])+abs(dfout['OUT_SPE']))
dfout['OUT_SPE_PERC_DELTA'] = dfout.groupby('Country')['OUT_SPE_PERC'].diff()
dfout['OUT_SPE_PERC_CHANGE'] = ((dfout['OUT_SPE'] - dfout.groupby('Country')['OUT_SPE'].shift(1)) / abs(dfout.groupby('Country')['OUT_SPE'].shift(1))) * 100
dfout['OUT_ALL_PERC_CHANGE'] = ((dfout['OUT_ALL'] - dfout.groupby('Country')['OUT_ALL'].shift(1)) / abs(dfout.groupby('Country')['OUT_ALL'].shift(1))) * 100
dfout = dfout.round(2)
dfout

,Country,Period,OUT_ALL,OUT_NON,OUT_SPE,OUT_SPE_PERC,OUT_SPE_PERC_DELTA,OUT_SPE_PERC_CHANGE,OUT_ALL_PERC_CHANGE
0,China,2015,29904.0,20998.0,8906.0,29.78,NaN,NaN,NaN
1,China,2016,30629.0,19782.0,10847.0,35.41,5.63,21.79,2.42
2,China,2017,26904.0,17604.0,9300.0,34.57,-0.85,-14.26,-12.16
3,China,2018,26671.0,16173.0,10498.0,39.36,4.79,12.88,-0.87
4,China,2019,40691.0,23988.0,16703.0,41.05,1.69,59.11,52.57
...,...,...,...,...,...,...,...,...,...
492,Total,2017,5068540.0,1990322.0,3078218.0,60.73,1.63,5.17,2.35
493,Total,2018,4985424.0,1913632.0,3071792.0,61.62,0.88,-0.21,-1.64
494,Total,2019,5135148.0,2013941.0,3121207.0,60.78,-0.83,1.61,3.00
495,Total,2020,4731164.0,2911108.0,1820056.0,38.47,-22.31,-41.69,-7.87


## Supplementary figures (GDP, MLI)

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRpVylytevUF6rfHB5s_xbKMouBQkIGE3y-79-MS_OmjYkMAHM1geldo9czUPZLJcYtEiou30EffwbF/pub?output=csv'
sup = pd.read_csv(url)
pd.options.display.float_format = '{:.2f}'.format
sup = pd.melt(sup, id_vars='Country', var_name='Period', value_name='GDP_US')
sup = sup.sort_values(['Country', 'Period']).reset_index(drop=True)
sup

,Country,Period,GDP_US
0,Afghanistan,2015,19998156214.40
1,Afghanistan,2016,18019558181.96
2,Afghanistan,2017,18896352021.94
3,Afghanistan,2018,18418848299.58
4,Afghanistan,2019,18904490262.91
...,...,...,...
1857,Zimbabwe,2017,17584890937.48
1858,Zimbabwe,2018,34156069918.06
1859,Zimbabwe,2019,21832234925.50
1860,Zimbabwe,2020,21509698406.11


In [ ]:
eur_usd = {2021: 1.18, 2020: 1.14, 2019: 1.12, 2018: 1.18, 2017: 1.13, 2016: 1.11, 2015: 1.11}
sup['GDP_EUR'] = sup.apply(lambda row: row['GDP_US'] / eur_usd[int(row['Period'])], axis=1)
sup

,Country,Period,GDP_US,GDP_EUR
0,Afghanistan,2015,19998156214.40,18016356949.91
1,Afghanistan,2016,18019558181.96,16233836199.96
2,Afghanistan,2017,18896352021.94,16722435417.65
3,Afghanistan,2018,18418848299.58,15609193474.22
4,Afghanistan,2019,18904490262.91,16879009163.32
...,...,...,...,...
1857,Zimbabwe,2017,17584890937.48,15561850387.15
1858,Zimbabwe,2018,34156069918.06,28945821964.46
1859,Zimbabwe,2019,21832234925.50,19493066897.77
1860,Zimbabwe,2020,21509698406.11,18868156496.59


In [ ]:
sup.drop('GDP_US', axis=1, inplace=True)

In [ ]:
sup['Country'].replace({'Hong Kong SAR, China': 'Hong Kong, China'}, inplace=True)
sup['Country'].replace({'Hong Kong': 'Hong Kong, China'}, inplace=True)
sup['Country'].replace({'Korea, Rep.': 'Korea, Republic of'}, inplace=True)
sup['Country'].replace({'Venezuela, RB': 'Venezuela, Bolivarian Republic'}, inplace=True)
sup['Country'].replace({'Czechia': 'Czech Republic'}, inplace=True)
sup['Country'].replace({'Slovak Republic': 'Slovakia'}, inplace=True)
sup['Country'].replace({'Egypt, Arab Rep.': 'Egypt'}, inplace=True)
sup['Country'].replace({'Turkiye': 'Turkey'}, inplace=True)
sup['Country'].replace({'World': 'Total'}, inplace=True)

In [ ]:
dfout['Period'] = dfout['Period'].astype(str)
dfin['Period'] = dfin['Period'].astype(str)
sup['Period'] = sup['Period'].astype(str)

In [ ]:
dfout = pd.merge(dfout, sup[['Country', 'Period', 'GDP_EUR']], on=['Country', 'Period'], how='left')
dfout.rename(columns={'GDP_EUR': 'GDP'}, inplace=True)
dfout['GDP'] = dfout['GDP']/1000000

In [ ]:
dfout['OUT_ALL_PERC_GDP'] = dfout['OUT_ALL']*100 / dfout['GDP']
dfout['OUT_SPE_PERC_GDP'] = dfout['OUT_SPE']*100 / dfout['GDP']
dfout

,Country,Period,OUT_ALL,OUT_NON,OUT_SPE,OUT_SPE_PERC,OUT_SPE_PERC_DELTA,OUT_SPE_PERC_CHANGE,OUT_ALL_PERC_CHANGE,GDP,OUT_ALL_PERC_GDP,OUT_SPE_PERC_GDP
0,China,2015,29904.00,20998.00,8906.00,29.78,NaN,NaN,NaN,9965363.14,0.30,0.09
1,China,2016,30629.00,19782.00,10847.00,35.41,5.63,21.79,2.42,10120068.95,0.30,0.11
2,China,2017,26904.00,17604.00,9300.00,34.57,-0.85,-14.26,-12.16,10894167.58,0.25,0.09
3,China,2018,26671.00,16173.00,10498.00,39.36,4.79,12.88,-0.87,11775269.11,0.23,0.09
4,China,2019,40691.00,23988.00,16703.00,41.05,1.69,59.11,52.57,12749944.20,0.32,0.13
...,...,...,...,...,...,...,...,...,...,...,...,...
492,Total,2017,5068540.00,1990322.00,3078218.00,60.73,1.63,5.17,2.35,72035991.36,7.04,4.27
493,Total,2018,4985424.00,1913632.00,3071792.00,61.62,0.88,-0.21,-1.64,73268549.37,6.80,4.19
494,Total,2019,5135148.00,2013941.00,3121207.00,60.78,-0.83,1.61,3.00,78254692.53,6.56,3.99
495,Total,2020,4731164.00,2911108.00,1820056.00,38.47,-22.31,-41.69,-7.87,74654038.25,6.34,2.44


In [ ]:
dfin = pd.merge(dfin, sup[['Country', 'Period', 'GDP_EUR']], on=['Country', 'Period'], how='left')
dfin.rename(columns={'GDP_EUR': 'GDP'}, inplace=True)
dfin['GDP'] = dfin['GDP']/1000000

In [ ]:
dfin['IN_ALL_PERC_GDP'] = dfin['IN_ALL']*100 / dfin['GDP']
dfin['IN_SPE_PERC_GDP'] = dfin['IN_SPE']*100 / dfin['GDP']
pd.options.display.float_format = '{:.2f}'.format
dfin

,Country,Period,IN_ALL,IN_NON,IN_SPE,IN_SPE_PERC,IN_SPE_PERC_DELTA,IN_SPE_PERC_CHANGE,IN_ALL_PERC_CHANGE,GDP,IN_ALL_PERC_GDP,IN_SPE_PERC_GDP
0,China,2015,20866.00,1843.00,19023.00,91.17,NaN,NaN,NaN,9965363.14,0.21,0.19
1,China,2016,18404.00,1458.00,16946.00,92.08,0.91,-10.92,-11.80,10120068.95,0.18,0.17
2,China,2017,17764.00,603.00,17161.00,96.61,4.53,1.27,-3.48,10894167.58,0.16,0.16
3,China,2018,18519.00,923.00,17596.00,95.02,-1.59,2.53,4.25,11775269.11,0.16,0.15
4,China,2019,17272.00,-802.00,18074.00,95.75,0.74,2.72,-6.73,12749944.20,0.14,0.14
...,...,...,...,...,...,...,...,...,...,...,...,...
492,Total,2017,4101393.00,1248788.00,2852605.00,69.55,1.19,5.29,3.48,72035991.36,5.69,3.96
493,Total,2018,3998868.00,1273992.00,2724876.00,68.14,-1.41,-4.48,-2.50,73268549.37,5.46,3.72
494,Total,2019,4055583.00,1270839.00,2784744.00,68.66,0.52,2.20,1.42,78254692.53,5.18,3.56
495,Total,2020,3687114.00,2217698.00,1469416.00,39.85,-28.81,-47.23,-9.09,74654038.25,4.94,1.97


In [ ]:
dfin['Period'] = dfin['Period'].astype(int)

mli_values = {
    'Australia': 2019,
    'Austria': 2018,
    'Belgium': 2019,
    'Canada': 2019,
    'Chile': 2021,
    'Croatia': 2021,
    'Cyprus': 2020,
    'Czech Republic': 2020,
    'Denmark': 2020,
    'Egypt': 2021,
    'Estonia': 2021,
    'Finland': 2019,
    'France': 2019,
    'Germany': 2021,
    'Greece': 2021,
    'Hungary': 2021,
    'Iceland': 2020,
    'India': 2019,
    'Indonesia': 2020,
    'Ireland': 2019,
    'Japan': 2019,
    'Korea, Republic of': 2020,
    'Latvia': 2020,
    'Liechtenstein': 2020,
    'Lithuania': 2019,
    'Luxembourg': 2019,
    'Malaysia': 2021,
    'Malta': 2019,
    'Mauritius': 2020,
    'New Zealand': 2018,
    'Norway': 2019,
    'Oman': 2020,
    'Panama': 2021,
    'Poland': 2018,
    'Portugal': 2020,
    'Russia': 2019,
    'Singapore': 2019,
    'Slovakia': 2019,
    'Slovenia': 2018,
    'Sweden': 2018,
    'Switzerland': 2019,
    'United Arab Emirates': 2019,
    'United Kingdom': 2018,
    'Uruguay': 2020
}

dfin['MLI'] = [
    1 if row['Country'] in mli_values and row['Period'] >= mli_values[row['Country']] else 0
    for _, row in dfin.iterrows()
]

dfin

,Country,Period,IN_ALL,IN_NON,IN_SPE,IN_SPE_PERC,IN_SPE_PERC_DELTA,IN_SPE_PERC_CHANGE,IN_ALL_PERC_CHANGE,GDP,IN_ALL_PERC_GDP,IN_SPE_PERC_GDP,MLI
0,China,2015,20866.00,1843.00,19023.00,91.17,NaN,NaN,NaN,9965363.14,0.21,0.19,0
1,China,2016,18404.00,1458.00,16946.00,92.08,0.91,-10.92,-11.80,10120068.95,0.18,0.17,0
2,China,2017,17764.00,603.00,17161.00,96.61,4.53,1.27,-3.48,10894167.58,0.16,0.16,0
3,China,2018,18519.00,923.00,17596.00,95.02,-1.59,2.53,4.25,11775269.11,0.16,0.15,0
4,China,2019,17272.00,-802.00,18074.00,95.75,0.74,2.72,-6.73,12749944.20,0.14,0.14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,Total,2017,4101393.00,1248788.00,2852605.00,69.55,1.19,5.29,3.48,72035991.36,5.69,3.96,0
493,Total,2018,3998868.00,1273992.00,2724876.00,68.14,-1.41,-4.48,-2.50,73268549.37,5.46,3.72,0
494,Total,2019,4055583.00,1270839.00,2784744.00,68.66,0.52,2.20,1.42,78254692.53,5.18,3.56,0
495,Total,2020,3687114.00,2217698.00,1469416.00,39.85,-28.81,-47.23,-9.09,74654038.25,4.94,1.97,0


In [ ]:
dfout['Period'] = dfin['Period'].astype(int)

dfout['MLI'] = [
    1 if row['Country'] in mli_values and row['Period'] >= mli_values[row['Country']] else 0
    for _, row in dfout.iterrows()
]

dfout

,Country,Period,OUT_ALL,OUT_NON,OUT_SPE,OUT_SPE_PERC,OUT_SPE_PERC_DELTA,OUT_SPE_PERC_CHANGE,OUT_ALL_PERC_CHANGE,GDP,OUT_ALL_PERC_GDP,OUT_SPE_PERC_GDP,MLI
0,China,2015,29904.00,20998.00,8906.00,29.78,NaN,NaN,NaN,9965363.14,0.30,0.09,0
1,China,2016,30629.00,19782.00,10847.00,35.41,5.63,21.79,2.42,10120068.95,0.30,0.11,0
2,China,2017,26904.00,17604.00,9300.00,34.57,-0.85,-14.26,-12.16,10894167.58,0.25,0.09,0
3,China,2018,26671.00,16173.00,10498.00,39.36,4.79,12.88,-0.87,11775269.11,0.23,0.09,0
4,China,2019,40691.00,23988.00,16703.00,41.05,1.69,59.11,52.57,12749944.20,0.32,0.13,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,Total,2017,5068540.00,1990322.00,3078218.00,60.73,1.63,5.17,2.35,72035991.36,7.04,4.27,0
493,Total,2018,4985424.00,1913632.00,3071792.00,61.62,0.88,-0.21,-1.64,73268549.37,6.80,4.19,0
494,Total,2019,5135148.00,2013941.00,3121207.00,60.78,-0.83,1.61,3.00,78254692.53,6.56,3.99,0
495,Total,2020,4731164.00,2911108.00,1820056.00,38.47,-22.31,-41.69,-7.87,74654038.25,6.34,2.44,0


In [ ]:
dnbin = dfin

dnbin = dnbin.rename(columns={'IN_ALL': 'ALL'})
dnbin = dnbin.rename(columns={'IN_SPE': 'SPE'})
dnbin = dnbin.rename(columns={'IN_SPE_PERC': 'SPE_SHARE'})
dnbin = dnbin.rename(columns={'IN_SPE_PERC_CHANGE': 'SPE_CHANGE'})
dnbin = dnbin.rename(columns={'IN_ALL_PERC_CHANGE': 'ALL_CHANGE'})
dnbin = dnbin.rename(columns={'IN_ALL_PERC_GDP': 'ALL/GDP'})
dnbin = dnbin.rename(columns={'IN_SPE_PERC_GDP': 'SPE/GDP'})
dnbin = dnbin.rename(columns={'IN_NON': 'NON_SPE'})
dnbin = dnbin.drop(['IN_SPE_PERC_DELTA'], axis=1)


in_change_col = dnbin.pop('ALL_CHANGE')
dnbin.insert(6, 'ALL_CHANGE', in_change_col)
gdp_col = dnbin.pop('GDP')
dnbin.insert(11, 'GDP', gdp_col)

dnbin

,Country,Period,ALL,NON_SPE,SPE,SPE_SHARE,ALL_CHANGE,SPE_CHANGE,ALL/GDP,SPE/GDP,MLI,GDP
0,China,2015,20866.00,1843.00,19023.00,91.17,NaN,NaN,0.21,0.19,0,9965363.14
1,China,2016,18404.00,1458.00,16946.00,92.08,-11.80,-10.92,0.18,0.17,0,10120068.95
2,China,2017,17764.00,603.00,17161.00,96.61,-3.48,1.27,0.16,0.16,0,10894167.58
3,China,2018,18519.00,923.00,17596.00,95.02,4.25,2.53,0.16,0.15,0,11775269.11
4,China,2019,17272.00,-802.00,18074.00,95.75,-6.73,2.72,0.14,0.14,0,12749944.20
...,...,...,...,...,...,...,...,...,...,...,...,...
492,Total,2017,4101393.00,1248788.00,2852605.00,69.55,3.48,5.29,5.69,3.96,0,72035991.36
493,Total,2018,3998868.00,1273992.00,2724876.00,68.14,-2.50,-4.48,5.46,3.72,0,73268549.37
494,Total,2019,4055583.00,1270839.00,2784744.00,68.66,1.42,2.20,5.18,3.56,0,78254692.53
495,Total,2020,3687114.00,2217698.00,1469416.00,39.85,-9.09,-47.23,4.94,1.97,0,74654038.25


In [ ]:
dnbout = dfout

dnbout = dnbout.rename(columns={'OUT_ALL': 'ALL'})
dnbout = dnbout.rename(columns={'OUT_SPE': 'SPE'})
dnbout = dnbout.rename(columns={'OUT_SPE_PERC': 'SPE_SHARE'})
dnbout = dnbout.rename(columns={'OUT_SPE_PERC_CHANGE': 'SPE_CHANGE'})
dnbout = dnbout.rename(columns={'OUT_ALL_PERC_CHANGE': 'ALL_CHANGE'})
dnbout = dnbout.rename(columns={'OUT_ALL_PERC_GDP': 'ALL/GDP'})
dnbout = dnbout.rename(columns={'OUT_SPE_PERC_GDP': 'SPE/GDP'})
dnbout = dnbout.rename(columns={'OUT_NON': 'NON_SPE'})
dnbout = dnbout.drop(['OUT_SPE_PERC_DELTA'], axis=1)

out_change_col = dnbout.pop('ALL_CHANGE')
dnbout.insert(6, 'ALL_CHANGE', out_change_col)
gdp_col = dnbout.pop('GDP')
dnbout.insert(11, 'GDP', gdp_col)

dnbout

,Country,Period,ALL,NON_SPE,SPE,SPE_SHARE,ALL_CHANGE,SPE_CHANGE,ALL/GDP,SPE/GDP,MLI,GDP
0,China,2015,29904.00,20998.00,8906.00,29.78,NaN,NaN,0.30,0.09,0,9965363.14
1,China,2016,30629.00,19782.00,10847.00,35.41,2.42,21.79,0.30,0.11,0,10120068.95
2,China,2017,26904.00,17604.00,9300.00,34.57,-12.16,-14.26,0.25,0.09,0,10894167.58
3,China,2018,26671.00,16173.00,10498.00,39.36,-0.87,12.88,0.23,0.09,0,11775269.11
4,China,2019,40691.00,23988.00,16703.00,41.05,52.57,59.11,0.32,0.13,0,12749944.20
...,...,...,...,...,...,...,...,...,...,...,...,...
492,Total,2017,5068540.00,1990322.00,3078218.00,60.73,2.35,5.17,7.04,4.27,0,72035991.36
493,Total,2018,4985424.00,1913632.00,3071792.00,61.62,-1.64,-0.21,6.80,4.19,0,73268549.37
494,Total,2019,5135148.00,2013941.00,3121207.00,60.78,3.00,1.61,6.56,3.99,0,78254692.53
495,Total,2020,4731164.00,2911108.00,1820056.00,38.47,-7.87,-41.69,6.34,2.44,0,74654038.25


## IMF Data

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRSHa3ruFMhl2QLkxEaaoBVprpSNiGJIzTBGZvB-iOH9txxE7ltTgCLJG1NugYc2J64b2XfsgonffXL/pub?output=csv'
in21 = pd.read_csv(url)
in21.insert(1, 'Year', 2021)
in21

,Country,Year,IN,IN_EQ,IN_NET
0,"Afghanistan, Islamic Rep. of",2021,0,3,-3
1,Albania,2021,1,1,0
2,Algeria,2021,39,-1,40
3,American Samoa,2021,0,0,0
4,"Andorra, Principality of",2021,1,2,-1
...,...,...,...,...,...
243,"Yemen, Rep. of",2021,1,1,0
244,Zambia,2021,-2,-2,0
245,Zimbabwe,2021,7,7,0
246,Not Specified (including Confidential),2021,NaN,NaN,NaN


In [ ]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQGGS7oxQQmYOVlZZZ8lrdKZcOC-jD7N0FlZ5xas2DhZwUPm_48pRhaHpHAE-95geKpoUpaQEkuR59l/pub?output=csv'
in20 = pd.read_csv(url)
in20.insert(1, 'Year', 2020)
in20

,Country,Year,IN,IN_EQ,IN_NET
0,"Afghanistan, Islamic Rep. of",2020,-2,2,-5
1,Albania,2020,2,2,0
2,Algeria,2020,79,-1,80
3,American Samoa,2020,0,0,0
4,"Andorra, Principality of",2020,1,2,-1
...,...,...,...,...,...
243,"Yemen, Rep. of",2020,1,1,0
244,Zambia,2020,1,1,0
245,Zimbabwe,2020,6,6,0
246,Not Specified (including Confidential),2020,NaN,NaN,NaN


In [ ]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTcNwywe7f8ZWcgrYXiApSB1JeGcZD17MJURIj9ZrRKYTPw1cIsN3C2QOZ7LlhL9OUaEU1OAmed8vgZ/pub?output=csv'
in19 = pd.read_csv(url)
in19.insert(1, 'Year', 2019)
in19

,Country,Year,IN,IN_EQ,IN_NET
0,"Afghanistan, Islamic Rep. of",2019,-4,2,-7
1,Albania,2019,2,3,-1
2,Algeria,2019,377,383,-6
3,American Samoa,2019,0,0,0
4,"Andorra, Principality of",2019,0,2,-2
...,...,...,...,...,...
243,"Yemen, Rep. of",2019,1,1,0
244,Zambia,2019,2,4,-2
245,Zimbabwe,2019,7,7,0
246,Not Specified (including Confidential),2019,NaN,NaN,NaN


In [ ]:
imfin = pd.concat([in19, in20, in21], ignore_index=True)
imfin = imfin.sort_values(by=['Country', 'Year'])
imfin = imfin.reset_index(drop=True)
imfin

,Country,Year,IN,IN_EQ,IN_NET
0,"Afghanistan, Islamic Rep. of",2019,-4,2,-7
1,"Afghanistan, Islamic Rep. of",2020,-2,2,-5
2,"Afghanistan, Islamic Rep. of",2021,0,3,-3
3,Albania,2019,2,3,-1
4,Albania,2020,2,2,0
...,...,...,...,...,...
739,Zambia,2020,1,1,0
740,Zambia,2021,-2,-2,0
741,Zimbabwe,2019,7,7,0
742,Zimbabwe,2020,6,6,0


In [ ]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRV3G81_vsA3Lx3RIyGwApHUZ60P6Jftckjp3OnVZro3mKNWl-f2hJz25bQCN-NuC2uYRtXgp0Zuhzn/pub?output=csv'
out21 = pd.read_csv(url)
out21.insert(1, 'Year', 2021)
out21

,Country,Year,OUT,OUT_EQ,OUT_NET
0,"Afghanistan, Islamic Rep. of",2021,-1 740,0,-1 740
1,Albania,2021,1 188,942,246
2,Algeria,2021,-3 331,-3 582,251
3,American Samoa,2021,40,52,-12
4,"Andorra, Principality of",2021,2,0,2
...,...,...,...,...,...
243,"Yemen, Rep. of",2021,1,1,0
244,Zambia,2021,1 044,331,714
245,Zimbabwe,2021,190,139,51
246,Not Specified (including Confidential),2021,NaN,NaN,NaN


In [ ]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTRqzREnaeO9c2KQ1ya15Mby_3HtH9z7fczTvCHJD-ipxu_6M6ps7hnHd4PtT-2BPRebhZGqnkUXP5N/pub?output=csv'
out20 = pd.read_csv(url)
out20.insert(1, 'Year', 2020)
out20

,Country,Year,OUT,OUT_EQ,OUT_NET
0,"Afghanistan, Islamic Rep. of",2020,10,0,10
1,Albania,2020,923,599,324
2,Algeria,2020,2 459,1 989,470
3,American Samoa,2020,71,71,0
4,"Andorra, Principality of",2020,28,28,0
...,...,...,...,...,...
243,"Yemen, Rep. of",2020,4,4,0
244,Zambia,2020,1 153,307,847
245,Zimbabwe,2020,163,131,32
246,Not Specified (including Confidential),2020,NaN,NaN,NaN


In [ ]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRKicJMyTDQOIOLyB58bLA75X_HcNXUZ7a_X8NmdOaIKhHGiK5C2ZQaltsO1MaLSc_V5Lyp__E3Q5zk/pub?output=csv'
out19 = pd.read_csv(url)
out19.insert(1, 'Year', 2019)
out19

,Country,Year,OUT,OUT_EQ,OUT_NET
0,"Afghanistan, Islamic Rep. of",2019,0,0,0
1,Albania,2019,900,621,279
2,Algeria,2019,122,97,26
3,American Samoa,2019,43,54,-11
4,"Andorra, Principality of",2019,10,10,0
...,...,...,...,...,...
243,"Yemen, Rep. of",2019,2,2,0
244,Zambia,2019,1 232,218,1 014
245,Zimbabwe,2019,151,120,30
246,Not Specified (including Confidential),2019,NaN,NaN,NaN


In [ ]:
imfout = pd.concat([out19, out20, out21], ignore_index=True)
imfout = imfout.sort_values(by=['Country', 'Year'])
imfout = imfout.reset_index(drop=True)
imfout

,Country,Year,OUT,OUT_EQ,OUT_NET
0,"Afghanistan, Islamic Rep. of",2019,0,0,0
1,"Afghanistan, Islamic Rep. of",2020,10,0,10
2,"Afghanistan, Islamic Rep. of",2021,-1 740,0,-1 740
3,Albania,2019,900,621,279
4,Albania,2020,923,599,324
...,...,...,...,...,...
739,Zambia,2020,1 153,307,847
740,Zambia,2021,1 044,331,714
741,Zimbabwe,2019,151,120,30
742,Zimbabwe,2020,163,131,32


In [ ]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vS30Y_uTuzrATmLDEKsyWzcuTIoOOtEIZ9QmdWFdhF5e9EL3jPkg137kZAQ9ZF-B9OoL9SnsuQd4nR5/pub?output=csv'
allin21 = pd.read_csv(url)
allin21.insert(1, 'Year', 2021)
allin21

,Country,Year,Total Investment
0,"Afghanistan, Islamic Rep. of",2021,-1 241
1,Albania,2021,7 367
2,Algeria,2021,18 181
3,American Samoa,2021,125
4,"Andorra, Principality of",2021,1 183
...,...,...,...
243,"Yemen, Rep. of",2021,755
244,Zambia,2021,8 291
245,Zimbabwe,2021,7 038
246,Not Specified (including Confidential),2021,1 212 285


In [ ]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSYT4Q5C3DcVu_bcDiS0JF1zr9mdDXA6Nn3TZLZXEv3BpAsSfgKH3bf-z5HD1FuoUFt_PHc7kO_hWkg/pub?output=csv'
allin20 = pd.read_csv(url)
allin20.insert(1, 'Year', 2020)
allin20

,Country,Year,Total Investment
0,"Afghanistan, Islamic Rep. of",2020,587
1,Albania,2020,7 714
2,Algeria,2020,26 406
3,American Samoa,2020,104
4,"Andorra, Principality of",2020,1 373
...,...,...,...
243,"Yemen, Rep. of",2020,860
244,Zambia,2020,9 001
245,Zimbabwe,2020,5 297
246,Not Specified (including Confidential),2020,1 234 562


In [ ]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRNTN6kPajeiCIz-vrh-ZRHxGmyFWnEQ3atYSN7G0d1skFaJ6CJ92i0UdKVJ5NHSks5m8STec3NJQ0f/pub?output=csv'
allin19 = pd.read_csv(url)
allin19.insert(1, 'Year', 2019)
allin19

,Country,Year,Total Investment
0,"Afghanistan, Islamic Rep. of",2019,601
1,Albania,2019,7 569
2,Algeria,2019,23 170
3,American Samoa,2019,94
4,"Andorra, Principality of",2019,1 192
...,...,...,...
243,"Yemen, Rep. of",2019,360
244,Zambia,2019,6 709
245,Zimbabwe,2019,5 330
246,Not Specified (including Confidential),2019,1 543 816


In [ ]:
allin = pd.concat([allin19, allin20, allin21], ignore_index=True)
allin = allin.sort_values(by=['Country', 'Year'])
allin = allin.reset_index(drop=True)
allin.rename(columns={'Total Investment': 'World DI'}, inplace=True)
allin

,Country,Year,World DI
0,"Afghanistan, Islamic Rep. of",2019,601
1,"Afghanistan, Islamic Rep. of",2020,587
2,"Afghanistan, Islamic Rep. of",2021,-1 241
3,Albania,2019,7 569
4,Albania,2020,7 714
...,...,...,...
739,Zambia,2020,9 001
740,Zambia,2021,8 291
741,Zimbabwe,2019,5 330
742,Zimbabwe,2020,5 297


In [ ]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTXP5j9PgWSQLNGb6doa4LGQL8Gy30c9APJVl-Dgudjpj_cdkhrHAP2hDqQnZl-gb6ixX2fWMgneY5O/pub?output=csv'
allout21 = pd.read_csv(url)
allout21.insert(1, 'Year', 2021)
allout21

,Country,Year,Total Investment
0,"Afghanistan, Islamic Rep. of",2021,920
1,Albania,2021,646
2,Algeria,2021,1 778
3,American Samoa,2021,5 456
4,"Andorra, Principality of",2021,1 748
...,...,...,...
243,"Yemen, Rep. of",2021,111
244,Zambia,2021,419
245,Zimbabwe,2021,1 318
246,Not Specified (including Confidential),2021,1 182 592


In [ ]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQ2oQ-yZs3m-RbKKvFvVVSB1PW24hrGTbk96uCB_Uu20mIBRZzUiXppa06cnldYkTHt5K4P89tBzhM6/pub?output=csv'
allout20 = pd.read_csv(url)
allout20.insert(1, 'Year', 2020)
allout20

,Country,Year,Total Investment
0,"Afghanistan, Islamic Rep. of",2020,989
1,Albania,2020,570
2,Algeria,2020,1 958
3,American Samoa,2020,5 391
4,"Andorra, Principality of",2020,1 854
...,...,...,...
243,"Yemen, Rep. of",2020,85
244,Zambia,2020,248
245,Zimbabwe,2020,1 361
246,Not Specified (including Confidential),2020,1 197 345


In [ ]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTAb-xsqByMctFQV5BuCzFY__dA8KTMrdTxWnzgYexrOBrUZeWlax39FpoxXmAUZUwb3Ft_BqDVcVxF/pub?output=csv'
allout19 = pd.read_csv(url)
allout19.insert(1, 'Year', 2019)
allout19

,Investment from:,Year,Total Investment
0,"Afghanistan, Islamic Rep. of",2019,759
1,Albania,2019,442
2,Algeria,2019,1 717
3,American Samoa,2019,4 989
4,"Andorra, Principality of",2019,1 320
...,...,...,...
243,"Yemen, Rep. of",2019,164
244,Zambia,2019,265
245,Zimbabwe,2019,1 929
246,Not Specified (including Confidential),2019,1 478 529


In [ ]:
allout = pd.concat([allout19, allout20, allout21], ignore_index=True)
allout = allin.sort_values(by=['Country', 'Year'])
allout = allin.reset_index(drop=True)
allout.rename(columns={'Total Investment': 'World DI'}, inplace=True)
allout

,Country,Year,World DI
0,"Afghanistan, Islamic Rep. of",2019,601
1,"Afghanistan, Islamic Rep. of",2020,587
2,"Afghanistan, Islamic Rep. of",2021,-1 241
3,Albania,2019,7 569
4,Albania,2020,7 714
...,...,...,...
739,Zambia,2020,9 001
740,Zambia,2021,8 291
741,Zimbabwe,2019,5 330
742,Zimbabwe,2020,5 297


In [ ]:
imfin = pd.merge(imfin, allout[['Country', 'Year', 'World DI']], on=['Country', 'Year'], how='left')
imfin

,Country,Year,IN,IN_EQ,IN_NET,World DI
0,"Afghanistan, Islamic Rep. of",2019,-4,2,-7,601
1,"Afghanistan, Islamic Rep. of",2020,-2,2,-5,587
2,"Afghanistan, Islamic Rep. of",2021,0,3,-3,-1 241
3,Albania,2019,2,3,-1,7 569
4,Albania,2020,2,2,0,7 714
...,...,...,...,...,...,...
739,Zambia,2020,1,1,0,9 001
740,Zambia,2021,-2,-2,0,8 291
741,Zimbabwe,2019,7,7,0,5 330
742,Zimbabwe,2020,6,6,0,5 297


In [ ]:
imfout = pd.merge(imfout, allin[['Country', 'Year', 'World DI']], on=['Country', 'Year'], how='left')
imfout

,Country,Year,OUT,OUT_EQ,OUT_NET,World DI
0,"Afghanistan, Islamic Rep. of",2019,0,0,0,601
1,"Afghanistan, Islamic Rep. of",2020,10,0,10,587
2,"Afghanistan, Islamic Rep. of",2021,-1 740,0,-1 740,-1 241
3,Albania,2019,900,621,279,7 569
4,Albania,2020,923,599,324,7 714
...,...,...,...,...,...,...
739,Zambia,2020,1 153,307,847,9 001
740,Zambia,2021,1 044,331,714,8 291
741,Zimbabwe,2019,151,120,30,5 330
742,Zimbabwe,2020,163,131,32,5 297


## Supplementary figures (GDP, MLI)

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRpVylytevUF6rfHB5s_xbKMouBQkIGE3y-79-MS_OmjYkMAHM1geldo9czUPZLJcYtEiou30EffwbF/pub?output=csv'
sup = pd.read_csv(url)
pd.options.display.float_format = '{:.2f}'.format
sup = pd.melt(sup, id_vars='Country', var_name='Year', value_name='GDP_US')
sup = sup.sort_values(['Country', 'Year']).reset_index(drop=True)
sup

,Country,Year,GDP_US
0,Afghanistan,2015,19998156214.40
1,Afghanistan,2016,18019558181.96
2,Afghanistan,2017,18896352021.94
3,Afghanistan,2018,18418848299.58
4,Afghanistan,2019,18904490262.91
...,...,...,...
1857,Zimbabwe,2017,17584890937.48
1858,Zimbabwe,2018,34156069918.06
1859,Zimbabwe,2019,21832234925.50
1860,Zimbabwe,2020,21509698406.11


In [ ]:
sup['Country'].replace({'Hong Kong SAR, China': 'Hong Kong, China'}, inplace=True)
sup['Country'].replace({'Hong Kong': 'Hong Kong, China'}, inplace=True)
sup['Country'].replace({'Korea, Rep.': 'Korea, Republic of'}, inplace=True)
sup['Country'].replace({'Venezuela, RB': 'Venezuela, Bolivarian Republic'}, inplace=True)
sup['Country'].replace({'Czechia': 'Czech Republic'}, inplace=True)
sup['Country'].replace({'Slovak Republic': 'Slovakia'}, inplace=True)
sup['Country'].replace({'Egypt, Arab Rep.': 'Egypt'}, inplace=True)
sup['Country'].replace({'Turkiye': 'Turkey'}, inplace=True)
sup['Country'].replace({'World': 'Total'}, inplace=True)
sup['Country'].replace({'Virgin Islands (U.S.)': 'United States Virgin Islands'}, inplace=True)

imfin['Country'].replace({'Afghanistan, Islamic Rep. of': 'Afghanistan'}, inplace=True)
imfout['Country'].replace({'Afghanistan, Islamic Rep. of': 'Afghanistan'}, inplace=True)
imfin['Country'].replace({'Andorra, Principality of': 'Andorra'},inplace=True)
imfout['Country'].replace({'Andorra, Principality of': 'Andorra'}, inplace=True)
imfin['Country'].replace({'Armenia, Rep. of': 'Armenia'},inplace=True)
imfout['Country'].replace({'Armenia, Rep. of': 'Armenia'}, inplace=True)
imfin['Country'].replace({'Aruba, Kingdom of the Netherlands': 'Aruba'},inplace=True)
imfout['Country'].replace({'Aruba, Kingdom of the Netherlands': 'Aruba'}, inplace=True)
imfin['Country'].replace({'Azerbaijan, Rep. of': 'Azerbaijan'},inplace=True)
imfout['Country'].replace({'Azerbaijan, Rep. of': 'Azerbaijan'}, inplace=True)
imfin['Country'].replace({'Bahrain, Kingdom of': 'Bahrain'},inplace=True)
imfout['Country'].replace({'Bahrain, Kingdom of': 'Bahrain'}, inplace=True)
imfin['Country'].replace({'Belarus, Rep. of': 'Belarus'},inplace=True)
imfout['Country'].replace({'Belarus, Rep. of': 'Belarus'}, inplace=True)
imfin['Country'].replace({'Central African Rep.': 'Central African Republic'},inplace=True)
imfout['Country'].replace({'Central African Rep.': 'Central African Republic'}, inplace=True)
imfin['Country'].replace({'China, P.R.: Hong Kong': 'Hong Kong, China'},inplace=True)
imfout['Country'].replace({'China, P.R.: Hong Kong': 'Hong Kong, China'},inplace=True)
imfin['Country'].replace({'China, P.R.: Mainland': 'China'},inplace=True)
imfout['Country'].replace({'China, P.R.: Mainland': 'China'}, inplace=True)
imfin['Country'].replace({'Comoros, Union of the': 'Comoros'},inplace=True)
imfout['Country'].replace({'Comoros, Union of the': 'Comoros'}, inplace=True)
imfin['Country'].replace({'Congo, Dem. Rep. of the': 'Congo, Dem. Rep.'},inplace=True)
imfout['Country'].replace({'Congo, Dem. Rep. of the': 'Congo, Dem. Rep.'},inplace=True)
imfin['Country'].replace({'Congo, Rep. of': 'Congo, Rep.'},inplace=True)
imfout['Country'].replace({'Congo, Rep. of': 'Congo, Rep.'}, inplace=True)
imfin['Country'].replace({'Croatia, Rep. of': 'Croatia'},inplace=True)
imfout['Country'].replace({'Croatia, Rep. of': 'Croatia'}, inplace=True)
imfin['Country'].replace({'Curaçao, Kingdom of the Netherlands': 'Curacao'},inplace=True)
imfout['Country'].replace({'Curaçao, Kingdom of the Netherlands': 'Curacao'}, inplace=True)
imfin['Country'].replace({'Czech Rep.': 'Czech Republic'},inplace=True)
imfout['Country'].replace({'Czech Rep.': 'Czech Republic'}, inplace=True)
imfin['Country'].replace({'Côte d\'Ivoire': 'Cote d\'Ivoire'},inplace=True)
imfout['Country'].replace({'Côte d\'Ivoire': 'Cote d\'Ivoire'}, inplace=True)
imfin['Country'].replace({'Dominican Rep.': 'Dominican Republic'},inplace=True)
imfout['Country'].replace({'Dominican Rep.': 'Dominican Republic'}, inplace=True)
imfin['Country'].replace({'Egypt, Arab Rep. of': 'Egypt'},inplace=True)
imfout['Country'].replace({'Egypt, Arab Rep. of': 'Egypt'}, inplace=True)
imfin['Country'].replace({'Equatorial Guinea, Rep. of': 'Equatorial Guinea'},inplace=True)
imfout['Country'].replace({'Equatorial Guinea, Rep. of': 'Equatorial Guinea'}, inplace=True)
imfin['Country'].replace({'Eritrea, The State of': 'Eritrea'},inplace=True)
imfout['Country'].replace({'Eritrea, The State of': 'Eritrea'}, inplace=True)
imfin['Country'].replace({'Estonia, Rep. of': 'Estonia'},inplace=True)
imfout['Country'].replace({'Estonia, Rep. of': 'Estonia'}, inplace=True)
imfin['Country'].replace({'Eswatini, Kingdom of': 'Eswatini'},inplace=True)
imfout['Country'].replace({'Eswatini, Kingdom of': 'Eswatini'}, inplace=True)
imfin['Country'].replace({'Ethiopia, The Federal Dem. Rep. of': 'Ethiopia'},inplace=True)
imfout['Country'].replace({'Ethiopia, The Federal Dem. Rep. of': 'Ethiopia'}, inplace=True)
imfin['Country'].replace({'Fiji, Rep. of': 'Fiji'},inplace=True)
imfout['Country'].replace({'Fiji, Rep. of': 'Fiji'}, inplace=True)
imfin['Country'].replace({'Guiana, French': 'Guyana'},inplace=True)
imfout['Country'].replace({'Guiana, French': 'Guyana'}, inplace=True)
imfin['Country'].replace({'Iran, Islamic Rep. of': 'Iran, Islamic Rep.'},inplace=True)
imfout['Country'].replace({'Iran, Islamic Rep. of': 'Iran, Islamic Rep.'}, inplace=True)
imfin['Country'].replace({'Kazakhstan, Rep. of': 'Kazakhstan'},inplace=True)
imfout['Country'].replace({'Kazakhstan, Rep. of': 'Kazakhstan'}, inplace=True)
imfin['Country'].replace({'Korea, Dem. People\'s Rep. of': 'Korea, Dem. People\'s Rep.'},inplace=True)
imfout['Country'].replace({'Korea, Dem. People\'s Rep. of': 'Korea, Dem. People\'s Rep.'}, inplace=True)
imfin['Country'].replace({'Korea, Rep. of': 'Korea, Republic of'},inplace=True)
imfout['Country'].replace({'Korea, Rep. of': 'Korea, Republic of'}, inplace=True)
imfin['Country'].replace({'Kosovo, Rep. of': 'Kosovo'},inplace=True)
imfout['Country'].replace({'Kosovo, Rep. of': 'Kosovo'}, inplace=True)
imfin['Country'].replace({'Kyrgyz Rep.': 'Kyrgyz Republic'},inplace=True)
imfout['Country'].replace({'Kyrgyz Rep.': 'Kyrgyz Republic'}, inplace=True)
imfin['Country'].replace({'Lao People\'s Dem. Rep.': 'Lao PDR'},inplace=True)
imfout['Country'].replace({'Lao People\'s Dem. Rep.': 'Lao PDR'}, inplace=True)
imfin['Country'].replace({'Lesotho, Kingdom of': 'Lesotho'},inplace=True)
imfout['Country'].replace({'Lesotho, Kingdom of': 'Lesotho'}, inplace=True)
imfin['Country'].replace({'Madagascar, Rep. of': 'Madagascar'},inplace=True)
imfout['Country'].replace({'Madagascar, Rep. of': 'Madagascar'}, inplace=True)
imfin['Country'].replace({'Marshall Islands, Rep. of the': 'Marshall Islands'},inplace=True)
imfout['Country'].replace({'Marshall Islands, Rep. of the': 'Marshall Islands'}, inplace=True)
imfin['Country'].replace({'Mauritania, Islamic Rep. of': 'Mauritania'},inplace=True)
imfout['Country'].replace({'Mauritania, Islamic Rep. of': 'Mauritania'}, inplace=True)
imfin['Country'].replace({'Moldova, Rep. of': 'Moldova'},inplace=True)
imfout['Country'].replace({'Moldova, Rep. of': 'Moldova'}, inplace=True)
imfin['Country'].replace({'Mozambique, Rep. of': 'Mozambique'},inplace=True)
imfout['Country'].replace({'Mozambique, Rep. of': 'Mozambique'}, inplace=True)
imfin['Country'].replace({'Nauru, Rep. of': 'Nauru'},inplace=True)
imfout['Country'].replace({'Nauru, Rep. of': 'Nauru'}, inplace=True)
imfin['Country'].replace({'Netherlands, The': 'Netherlands'},inplace=True)
imfout['Country'].replace({'Netherlands, The': 'Netherlands'}, inplace=True)
imfin['Country'].replace({'North Macedonia, Republic of': 'North Macedonia'},inplace=True)
imfout['Country'].replace({'North Macedonia, Republic of': 'North Macedonia'}, inplace=True)
imfin['Country'].replace({'Palau, Rep. of': 'Palau'},inplace=True)
imfout['Country'].replace({'Palau, Rep. of': 'Palau'}, inplace=True)
imfin['Country'].replace({'Poland, Rep. of': 'Poland'},inplace=True)
imfout['Country'].replace({'Poland, Rep. of': 'Poland'}, inplace=True)
imfin['Country'].replace({'San Marino, Rep. of': 'San Marino'},inplace=True)
imfout['Country'].replace({'San Marino, Rep. of': 'San Marino'}, inplace=True)
imfin['Country'].replace({'Serbia, Rep. of': 'Serbia'},inplace=True)
imfout['Country'].replace({'Serbia, Rep. of': 'Serbia'}, inplace=True)
imfin['Country'].replace({'Slovak Rep.': 'Slovakia'},inplace=True)
imfout['Country'].replace({'Slovak Rep.': 'Slovakia'}, inplace=True)
imfin['Country'].replace({'Slovenia, Rep. of': 'Slovenia'},inplace=True)
imfout['Country'].replace({'Slovenia, Rep. of': 'Slovenia'}, inplace=True)
imfin['Country'].replace({'South Sudan, Rep. of': 'South Sudan'},inplace=True)
imfout['Country'].replace({'South Sudan, Rep. of': 'South Sudan'}, inplace=True)
imfin['Country'].replace({'Syrian Arab Rep.': 'Syrian Arab Republic'},inplace=True)
imfout['Country'].replace({'Syrian Arab Rep.': 'Syrian Arab Republic'}, inplace=True)
imfin['Country'].replace({'São Tomé and Príncipe, Dem. Rep. of': 'Sao Tome and Principe'},inplace=True)
imfout['Country'].replace({'São Tomé and Príncipe, Dem. Rep. of': 'Sao Tome and Principe'}, inplace=True)
imfin['Country'].replace({'Tajikistan, Rep. of': 'Tajikistan'},inplace=True)
imfout['Country'].replace({'Tajikistan, Rep. of': 'Tajikistan'}, inplace=True)
imfin['Country'].replace({'Tanzania, United Rep. of': 'Tanzania'},inplace=True)
imfout['Country'].replace({'Tanzania, United Rep. of': 'Tanzania'}, inplace=True)
imfin['Country'].replace({'Timor-Leste, Dem. Rep. of': 'Timor-Leste'},inplace=True)
imfout['Country'].replace({'Timor-Leste, Dem. Rep. of': 'Timor-Leste'}, inplace=True)
imfin['Country'].replace({'Türkiye, Rep of': 'Turkey'},inplace=True)
imfout['Country'].replace({'Türkiye, Rep of': 'Turkey'}, inplace=True)
imfin['Country'].replace({'Uzbekistan, Rep. of': 'Uzbekistan'},inplace=True)
imfout['Country'].replace({'Uzbekistan, Rep. of': 'Uzbekistan'}, inplace=True)
imfin['Country'].replace({'Venezuela, Rep. Bolivariana de': 'Venezuela, Bolivarian Republic'},inplace=True)
imfout['Country'].replace({'Venezuela, Rep. Bolivariana de': 'Venezuela, Bolivarian Republic'}, inplace=True)
imfin['Country'].replace({'Yemen, Rep. of': 'Yemen'},inplace=True)
imfout['Country'].replace({'Yemen, Rep. of': 'Yemen'}, inplace=True)
imfin['Country'].replace({'	China, P.R.: Macao': 'Macao SAR, China'},inplace=True)
imfout['Country'].replace({'China, P.R.: Macao': 'Macao SAR, China'}, inplace=True)
imfin['Country'].replace({'Micronesia, Federated States of': 'Micronesia, Fed. Sts.'},inplace=True)
imfout['Country'].replace({'Micronesia, Federated States of': 'Micronesia, Fed. Sts.'}, inplace=True)
imfin['Country'].replace({'Sint Maarten, Kingdom of the Netherlands': 'Sint Maarten (Dutch part)'},inplace=True)
imfout['Country'].replace({'Sint Maarten, Kingdom of the Netherlands': 'Sint Maarten (Dutch part)'}, inplace=True)
imfin['Country'].replace({'World': 'Total'},inplace=True)
imfout['Country'].replace({'World': 'Total'}, inplace=True)

In [ ]:
imfin['Year'] = imfin['Year'].astype(str)
imfout['Year'] = imfout['Year'].astype(str)
sup['Year'] = sup['Year'].astype(str)

In [ ]:
imfin = pd.merge(imfin, sup[['Country', 'Year', 'GDP_US']], on=['Country', 'Year'], how='left')
imfin.rename(columns={'GDP_US': 'GDP'}, inplace=True)
imfin['GDP'] = imfin['GDP']/1000000
imfin

,Country,Year,IN,IN_EQ,IN_NET,World DI,GDP
0,Afghanistan,2019,-4,2,-7,601,18904.49
1,Afghanistan,2020,-2,2,-5,587,20143.44
2,Afghanistan,2021,0,3,-3,-1 241,14786.86
3,Albania,2019,2,3,-1,7 569,15401.83
4,Albania,2020,2,2,0,7 714,15131.87
...,...,...,...,...,...,...,...
739,Zambia,2020,1,1,0,9 001,18110.63
740,Zambia,2021,-2,-2,0,8 291,22147.63
741,Zimbabwe,2019,7,7,0,5 330,21832.23
742,Zimbabwe,2020,6,6,0,5 297,21509.70


In [ ]:
imfout = pd.merge(imfout, sup[['Country', 'Year', 'GDP_US']], on=['Country', 'Year'], how='left')
imfout.rename(columns={'GDP_US': 'GDP'}, inplace=True)
imfout['GDP'] = imfout['GDP']/1000000
imfout

,Country,Year,OUT,OUT_EQ,OUT_NET,World DI,GDP
0,Afghanistan,2019,0,0,0,601,18904.49
1,Afghanistan,2020,10,0,10,587,20143.44
2,Afghanistan,2021,-1 740,0,-1 740,-1 241,14786.86
3,Albania,2019,900,621,279,7 569,15401.83
4,Albania,2020,923,599,324,7 714,15131.87
...,...,...,...,...,...,...,...
739,Zambia,2020,1 153,307,847,9 001,18110.63
740,Zambia,2021,1 044,331,714,8 291,22147.63
741,Zimbabwe,2019,151,120,30,5 330,21832.23
742,Zimbabwe,2020,163,131,32,5 297,21509.70


In [ ]:
imfin['Year'] = imfin['Year'].astype(int)

imfin['MLI'] = [
    1 if row['Country'] in mli_values and row['Year'] >= mli_values[row['Country']] else 0
    for _, row in imfin.iterrows()
]

imfin

,Country,Year,IN,IN_EQ,IN_NET,World DI,GDP,MLI
0,Afghanistan,2019,-4,2,-7,601,18904.49,0
1,Afghanistan,2020,-2,2,-5,587,20143.44,0
2,Afghanistan,2021,0,3,-3,-1 241,14786.86,0
3,Albania,2019,2,3,-1,7 569,15401.83,0
4,Albania,2020,2,2,0,7 714,15131.87,0
...,...,...,...,...,...,...,...,...
739,Zambia,2020,1,1,0,9 001,18110.63,0
740,Zambia,2021,-2,-2,0,8 291,22147.63,0
741,Zimbabwe,2019,7,7,0,5 330,21832.23,0
742,Zimbabwe,2020,6,6,0,5 297,21509.70,0


In [ ]:
imfout['Year'] = imfout['Year'].astype(int)

imfout['MLI'] = [
    1 if row['Country'] in mli_values and row['Year'] >= mli_values[row['Country']] else 0
    for _, row in imfout.iterrows()
]

imfout

,Country,Year,OUT,OUT_EQ,OUT_NET,World DI,GDP,MLI
0,Afghanistan,2019,0,0,0,601,18904.49,0
1,Afghanistan,2020,10,0,10,587,20143.44,0
2,Afghanistan,2021,-1 740,0,-1 740,-1 241,14786.86,0
3,Albania,2019,900,621,279,7 569,15401.83,0
4,Albania,2020,923,599,324,7 714,15131.87,0
...,...,...,...,...,...,...,...,...
739,Zambia,2020,1 153,307,847,9 001,18110.63,0
740,Zambia,2021,1 044,331,714,8 291,22147.63,0
741,Zimbabwe,2019,151,120,30,5 330,21832.23,0
742,Zimbabwe,2020,163,131,32,5 297,21509.70,0


In [ ]:
imfin = imfin.rename(columns={'IN':'DI'})
imfin = imfin.rename(columns={'IN_EQ':'EQUITY'})
imfin = imfin.rename(columns={'IN_NET':'NET'})
imfin = imfin.rename(columns={'Year':'Period'})
imfin = imfin.rename(columns={'World DI':'WDI'})
imfin

,Country,Period,DI,EQUITY,NET,WDI,GDP,MLI
0,Afghanistan,2019,-4,2,-7,601,18904.49,0
1,Afghanistan,2020,-2,2,-5,587,20143.44,0
2,Afghanistan,2021,0,3,-3,-1 241,14786.86,0
3,Albania,2019,2,3,-1,7 569,15401.83,0
4,Albania,2020,2,2,0,7 714,15131.87,0
...,...,...,...,...,...,...,...,...
739,Zambia,2020,1,1,0,9 001,18110.63,0
740,Zambia,2021,-2,-2,0,8 291,22147.63,0
741,Zimbabwe,2019,7,7,0,5 330,21832.23,0
742,Zimbabwe,2020,6,6,0,5 297,21509.70,0


In [ ]:
imfout = imfout.rename(columns={'OUT':'DI'})
imfout = imfout.rename(columns={'OUT_EQ':'EQUITY'})
imfout = imfout.rename(columns={'OUT_NET':'NET'})
imfout = imfout.rename(columns={'Year':'Period'})
imfout = imfout.rename(columns={'World DI':'WDI'})
imfout

,Country,Period,DI,EQUITY,NET,WDI,GDP,MLI
0,Afghanistan,2019,0,0,0,601,18904.49,0
1,Afghanistan,2020,10,0,10,587,20143.44,0
2,Afghanistan,2021,-1 740,0,-1 740,-1 241,14786.86,0
3,Albania,2019,900,621,279,7 569,15401.83,0
4,Albania,2020,923,599,324,7 714,15131.87,0
...,...,...,...,...,...,...,...,...
739,Zambia,2020,1 153,307,847,9 001,18110.63,0
740,Zambia,2021,1 044,331,714,8 291,22147.63,0
741,Zimbabwe,2019,151,120,30,5 330,21832.23,0
742,Zimbabwe,2020,163,131,32,5 297,21509.70,0


In [ ]:
imfout['DI'] = imfout['DI'].apply(lambda x: float(str(x).replace('\xa0','')))
imfout['EQUITY'] = imfout['EQUITY'].apply(lambda x: float(str(x).replace('\xa0','')))
imfout['NET'] = imfout['NET'].apply(lambda x: float(str(x).replace('\xa0','')))
imfout['WDI'] = imfout['WDI'].apply(lambda x: float(str(x).replace('\xa0','')))
imfout

,Country,Period,DI,EQUITY,NET,WDI,GDP,MLI
0,Afghanistan,2019,0.00,0.00,0.00,601.00,18904.49,0
1,Afghanistan,2020,10.00,0.00,10.00,587.00,20143.44,0
2,Afghanistan,2021,-1740.00,0.00,-1740.00,-1241.00,14786.86,0
3,Albania,2019,900.00,621.00,279.00,7569.00,15401.83,0
4,Albania,2020,923.00,599.00,324.00,7714.00,15131.87,0
...,...,...,...,...,...,...,...,...
739,Zambia,2020,1153.00,307.00,847.00,9001.00,18110.63,0
740,Zambia,2021,1044.00,331.00,714.00,8291.00,22147.63,0
741,Zimbabwe,2019,151.00,120.00,30.00,5330.00,21832.23,0
742,Zimbabwe,2020,163.00,131.00,32.00,5297.00,21509.70,0


In [ ]:
imfin['DI'] = imfin['DI'].apply(lambda x: float(str(x).replace('\xa0','')))
imfin['EQUITY'] = imfin['EQUITY'].apply(lambda x: float(str(x).replace('\xa0','')))
imfin['NET'] = imfin['NET'].apply(lambda x: float(str(x).replace('\xa0','')))
imfin['WDI'] = imfin['WDI'].apply(lambda x: float(str(x).replace('\xa0','')))
imfin

,Country,Period,DI,EQUITY,NET,WDI,GDP,MLI
0,Afghanistan,2019,-4.00,2.00,-7.00,601.00,18904.49,0
1,Afghanistan,2020,-2.00,2.00,-5.00,587.00,20143.44,0
2,Afghanistan,2021,0.00,3.00,-3.00,-1241.00,14786.86,0
3,Albania,2019,2.00,3.00,-1.00,7569.00,15401.83,0
4,Albania,2020,2.00,2.00,0.00,7714.00,15131.87,0
...,...,...,...,...,...,...,...,...
739,Zambia,2020,1.00,1.00,0.00,9001.00,18110.63,0
740,Zambia,2021,-2.00,-2.00,0.00,8291.00,22147.63,0
741,Zimbabwe,2019,7.00,7.00,0.00,5330.00,21832.23,0
742,Zimbabwe,2020,6.00,6.00,0.00,5297.00,21509.70,0


In [ ]:
imfin['DI_CHANGE'] = ((imfin['DI'] - imfin.groupby('Country')['DI'].shift(1)) / abs(imfin.groupby('Country')['DI'].shift(1))) * 100
imfin['DI/GDP'] = imfin['DI']*100 / imfin['GDP']
imfin['DI/WDI'] = imfin['DI']*100 / imfin['WDI']
imfin

,Country,Period,DI,EQUITY,NET,WDI,GDP,MLI,DI_CHANGE,DI/GDP,DI/WDI
0,Afghanistan,2019,-4.00,2.00,-7.00,601.00,18904.49,0,NaN,-0.02,-0.67
1,Afghanistan,2020,-2.00,2.00,-5.00,587.00,20143.44,0,50.00,-0.01,-0.34
2,Afghanistan,2021,0.00,3.00,-3.00,-1241.00,14786.86,0,100.00,0.00,-0.00
3,Albania,2019,2.00,3.00,-1.00,7569.00,15401.83,0,NaN,0.01,0.03
4,Albania,2020,2.00,2.00,0.00,7714.00,15131.87,0,0.00,0.01,0.03
...,...,...,...,...,...,...,...,...,...,...,...
739,Zambia,2020,1.00,1.00,0.00,9001.00,18110.63,0,-50.00,0.01,0.01
740,Zambia,2021,-2.00,-2.00,0.00,8291.00,22147.63,0,-300.00,-0.01,-0.02
741,Zimbabwe,2019,7.00,7.00,0.00,5330.00,21832.23,0,NaN,0.03,0.13
742,Zimbabwe,2020,6.00,6.00,0.00,5297.00,21509.70,0,-14.29,0.03,0.11


In [ ]:
imfout['DI_CHANGE'] = ((imfout['DI'] - imfout.groupby('Country')['DI'].shift(1)) / abs(imfout.groupby('Country')['DI'].shift(1))) * 100
imfout['DI/GDP'] = imfout['DI']*100 / imfout['GDP']
imfout['DI/WDI'] = imfout['DI']*100 / imfout['WDI']
imfout

,Country,Period,DI,EQUITY,NET,WDI,GDP,MLI,DI_CHANGE,DI/GDP,DI/WDI
0,Afghanistan,2019,0.00,0.00,0.00,601.00,18904.49,0,NaN,0.00,0.00
1,Afghanistan,2020,10.00,0.00,10.00,587.00,20143.44,0,inf,0.05,1.70
2,Afghanistan,2021,-1740.00,0.00,-1740.00,-1241.00,14786.86,0,-17500.00,-11.77,140.21
3,Albania,2019,900.00,621.00,279.00,7569.00,15401.83,0,NaN,5.84,11.89
4,Albania,2020,923.00,599.00,324.00,7714.00,15131.87,0,2.56,6.10,11.97
...,...,...,...,...,...,...,...,...,...,...,...
739,Zambia,2020,1153.00,307.00,847.00,9001.00,18110.63,0,-6.41,6.37,12.81
740,Zambia,2021,1044.00,331.00,714.00,8291.00,22147.63,0,-9.45,4.71,12.59
741,Zimbabwe,2019,151.00,120.00,30.00,5330.00,21832.23,0,NaN,0.69,2.83
742,Zimbabwe,2020,163.00,131.00,32.00,5297.00,21509.70,0,7.95,0.76,3.08


In [ ]:
di = imfin.pop('DI_CHANGE')
imfin.insert(3, 'DI_CHANGE', di)
digdp = imfin.pop('DI/GDP')
imfin.insert(4, 'DI/GDP', digdp)
diwdi = imfin.pop('DI/WDI')
imfin.insert(5, 'DI/WDI',diwdi)
mli = imfin.pop('MLI')
imfin.insert(6, 'MLI', mli)
gdp = imfin.pop('GDP')
imfin.insert(7, 'GDP', gdp)

imfin

,Country,Period,DI,DI_CHANGE,DI/GDP,DI/WDI,MLI,GDP,EQUITY,NET,WDI
0,Afghanistan,2019,-4.00,NaN,-0.02,-0.67,0,18904.49,2.00,-7.00,601.00
1,Afghanistan,2020,-2.00,50.00,-0.01,-0.34,0,20143.44,2.00,-5.00,587.00
2,Afghanistan,2021,0.00,100.00,0.00,-0.00,0,14786.86,3.00,-3.00,-1241.00
3,Albania,2019,2.00,NaN,0.01,0.03,0,15401.83,3.00,-1.00,7569.00
4,Albania,2020,2.00,0.00,0.01,0.03,0,15131.87,2.00,0.00,7714.00
...,...,...,...,...,...,...,...,...,...,...,...
739,Zambia,2020,1.00,-50.00,0.01,0.01,0,18110.63,1.00,0.00,9001.00
740,Zambia,2021,-2.00,-300.00,-0.01,-0.02,0,22147.63,-2.00,0.00,8291.00
741,Zimbabwe,2019,7.00,NaN,0.03,0.13,0,21832.23,7.00,0.00,5330.00
742,Zimbabwe,2020,6.00,-14.29,0.03,0.11,0,21509.70,6.00,0.00,5297.00


In [ ]:
di = imfout.pop('DI_CHANGE')
imfout.insert(3, 'DI_CHANGE', di)
digdp = imfout.pop('DI/GDP')
imfout.insert(4, 'DI/GDP', digdp)
diwdi = imfout.pop('DI/WDI')
imfout.insert(5, 'DI/WDI',diwdi)
mli = imfout.pop('MLI')
imfout.insert(6, 'MLI', mli)
gdp = imfout.pop('GDP')
imfout.insert(7, 'GDP', gdp)

imfout

,Country,Period,DI,DI_CHANGE,DI/GDP,DI/WDI,MLI,GDP,EQUITY,NET,WDI
0,Afghanistan,2019,0.00,NaN,0.00,0.00,0,18904.49,0.00,0.00,601.00
1,Afghanistan,2020,10.00,inf,0.05,1.70,0,20143.44,0.00,10.00,587.00
2,Afghanistan,2021,-1740.00,-17500.00,-11.77,140.21,0,14786.86,0.00,-1740.00,-1241.00
3,Albania,2019,900.00,NaN,5.84,11.89,0,15401.83,621.00,279.00,7569.00
4,Albania,2020,923.00,2.56,6.10,11.97,0,15131.87,599.00,324.00,7714.00
...,...,...,...,...,...,...,...,...,...,...,...
739,Zambia,2020,1153.00,-6.41,6.37,12.81,0,18110.63,307.00,847.00,9001.00
740,Zambia,2021,1044.00,-9.45,4.71,12.59,0,22147.63,331.00,714.00,8291.00
741,Zimbabwe,2019,151.00,NaN,0.69,2.83,0,21832.23,120.00,30.00,5330.00
742,Zimbabwe,2020,163.00,7.95,0.76,3.08,0,21509.70,131.00,32.00,5297.00


## Clean

In [ ]:
imfin = imfin.drop(['EQUITY', 'NET'], axis=1)
imfout = imfout.drop(['EQUITY', 'NET'], axis=1)

In [ ]:
dnbin[dnbin['Period'].isin([2020, 2021])]

,Country,Period,ALL,NON_SPE,SPE,SPE_SHARE,ALL_CHANGE,SPE_CHANGE,ALL/GDP,SPE/GDP,MLI,GDP
5,China,2020,20206.00,2856.00,17350.00,85.87,16.99,-4.01,0.16,0.13,0,12883924.47
6,China,2021,21908.00,692.00,21216.00,96.84,8.42,22.28,0.15,0.14,0,15028866.65
12,"Hong Kong, China",2020,60389.00,41429.00,18960.00,31.40,-36.91,-72.21,19.96,6.27,0,302572.10
13,"Hong Kong, China",2021,58657.00,41248.00,17409.00,29.68,-2.87,-8.18,18.75,5.56,0,312861.36
19,India,2020,3393.00,-457.00,3850.00,89.39,-14.88,-18.47,0.14,0.16,1,2340077.15
...,...,...,...,...,...,...,...,...,...,...,...,...
482,Turkey,2021,13905.00,3412.00,10493.00,75.46,3.48,4.45,2.00,1.51,0,694097.61
488,United Kingdom,2020,368727.00,185186.00,183541.00,49.78,-7.40,-30.17,15.54,7.74,1,2372464.18
489,United Kingdom,2021,544299.00,404815.00,139484.00,25.63,47.62,-24.00,20.51,5.26,1,2653709.97
495,Total,2020,3687114.00,2217698.00,1469416.00,39.85,-9.09,-47.23,4.94,1.97,0,74654038.25


In [ ]:
imfin[imfin['Period']!=2019]

,Country,Period,DI,DI_CHANGE,DI/GDP,DI/WDI,MLI,GDP,WDI
1,Afghanistan,2020,-2.00,50.00,-0.01,-0.34,0,20143.44,587.00
2,Afghanistan,2021,0.00,100.00,0.00,-0.00,0,14786.86,-1241.00
4,Albania,2020,2.00,0.00,0.01,0.03,0,15131.87,7714.00
5,Albania,2021,1.00,-50.00,0.01,0.01,0,18255.79,7367.00
7,Algeria,2020,79.00,-79.05,0.05,0.30,0,145009.18,26406.00
...,...,...,...,...,...,...,...,...,...
737,Yemen,2021,1.00,0.00,NaN,0.13,0,NaN,755.00
739,Zambia,2020,1.00,-50.00,0.01,0.01,0,18110.63,9001.00
740,Zambia,2021,-2.00,-300.00,-0.01,-0.02,0,22147.63,8291.00
742,Zimbabwe,2020,6.00,-14.29,0.03,0.11,0,21509.70,5297.00


In [ ]:
dnbout[dnbout['Period'].isin([2020, 2021])]

,Country,Period,ALL,NON_SPE,SPE,SPE_SHARE,ALL_CHANGE,SPE_CHANGE,ALL/GDP,SPE/GDP,MLI,GDP
5,China,2020,46372.00,33472.00,12900.00,27.82,13.96,-22.77,0.36,0.10,0,12883924.47
6,China,2021,44389.00,31831.00,12558.00,28.29,-4.28,-2.65,0.30,0.08,0,15028866.65
12,"Hong Kong, China",2020,26780.00,20325.00,6455.00,24.10,-43.13,-49.52,8.85,2.13,0,302572.10
13,"Hong Kong, China",2021,25859.00,18777.00,7082.00,27.39,-3.44,9.71,8.27,2.26,0,312861.36
19,India,2020,39167.00,29039.00,10128.00,25.86,16.11,-48.66,1.67,0.43,1,2340077.15
...,...,...,...,...,...,...,...,...,...,...,...,...
482,Turkey,2021,15803.00,9503.00,6300.00,39.87,-10.60,-2.25,2.28,0.91,0,694097.61
488,United Kingdom,2020,600144.00,441821.00,158323.00,26.38,-5.22,-56.71,25.30,6.67,1,2372464.18
489,United Kingdom,2021,708984.00,549849.00,159135.00,22.45,18.14,0.51,26.72,6.00,1,2653709.97
495,Total,2020,4731164.00,2911108.00,1820056.00,38.47,-7.87,-41.69,6.34,2.44,0,74654038.25


In [ ]:
imfout[imfout['Period']!=2019]

,Country,Period,DI,DI_CHANGE,DI/GDP,DI/WDI,MLI,GDP,WDI
1,Afghanistan,2020,10.00,inf,0.05,1.70,0,20143.44,587.00
2,Afghanistan,2021,-1740.00,-17500.00,-11.77,140.21,0,14786.86,-1241.00
4,Albania,2020,923.00,2.56,6.10,11.97,0,15131.87,7714.00
5,Albania,2021,1188.00,28.71,6.51,16.13,0,18255.79,7367.00
7,Algeria,2020,2459.00,1915.57,1.70,9.31,0,145009.18,26406.00
...,...,...,...,...,...,...,...,...,...
737,Yemen,2021,1.00,-75.00,NaN,0.13,0,NaN,755.00
739,Zambia,2020,1153.00,-6.41,6.37,12.81,0,18110.63,9001.00
740,Zambia,2021,1044.00,-9.45,4.71,12.59,0,22147.63,8291.00
742,Zimbabwe,2020,163.00,7.95,0.76,3.08,0,21509.70,5297.00


# Analysis

## Inward Foreign Direct Investment

### DNB

*Context*

In [ ]:
dnbin[(dnbin['Country'] == 'Total')]

,Country,Period,ALL,NON_SPE,SPE,SPE_SHARE,ALL_CHANGE,SPE_CHANGE,ALL/GDP,SPE/GDP,MLI,GDP
490,Total,2015,3623737.00,1170941.00,2452796.00,67.69,NaN,NaN,5.35,3.62,0,67736794.45
491,Total,2016,3963282.00,1253906.00,2709376.00,68.36,9.37,10.46,5.75,3.93,0,68884276.35
492,Total,2017,4101393.00,1248788.00,2852605.00,69.55,3.48,5.29,5.69,3.96,0,72035991.36
493,Total,2018,3998868.00,1273992.00,2724876.00,68.14,-2.50,-4.48,5.46,3.72,0,73268549.37
494,Total,2019,4055583.00,1270839.00,2784744.00,68.66,1.42,2.20,5.18,3.56,0,78254692.53
495,Total,2020,3687114.00,2217698.00,1469416.00,39.85,-9.09,-47.23,4.94,1.97,0,74654038.25
496,Total,2021,3824322.00,2423147.00,1401175.00,36.64,3.72,-4.64,4.68,1.71,0,81790743.53


*Countries with the largest **increase** in SPE investment into the 
Netherlands*

*2020*

In [ ]:
dnbin_2020 = dnbin[dnbin['Period']==2020]
dnbin_2020.sort_values(by='SPE_CHANGE', ascending=False).head(10)

,Country,Period,ALL,NON_SPE,SPE,SPE_SHARE,ALL_CHANGE,SPE_CHANGE,ALL/GDP,SPE/GDP,MLI,GDP
418,Nigeria,2020,1250.00,72.00,1178.00,94.24,348.03,388.80,0.33,0.31,0,379121.87
320,Romania,2020,566.00,55.00,511.00,90.28,150.44,146.86,0.26,0.23,0,220493.02
187,Croatia,2020,-114.00,-137.00,23.00,14.38,-660.00,130.00,-0.23,0.05,0,50414.05
75,Singapore,2020,102252.00,13977.00,88275.00,86.33,48.29,32.63,33.76,29.14,1,302891.17
439,Iceland,2020,19391.00,2012.00,17379.00,89.62,22.48,18.89,101.89,91.32,1,19030.42
271,Latvia,2020,-8.00,11.00,-19.00,-63.33,63.64,13.64,-0.03,-0.06,1,30352.40
250,Hungary,2020,14224.00,13021.00,1203.00,8.46,570.31,8.97,10.32,0.87,0,137878.99
26,Indonesia,2020,1497.00,1003.00,494.00,33.00,270.54,7.16,0.16,0.05,1,928674.50
404,Morocco,2020,39.00,-7.00,46.00,86.79,550.00,2.22,0.04,0.04,0,106445.74
68,Philippines,2020,602.00,95.00,507.00,84.22,-5.49,-3.61,0.19,0.16,0,317325.54


*2021*

In [ ]:
dnbin_2021 = dnbin[dnbin['Period']==2021]
dnbin_2021.sort_values(by='SPE_CHANGE', ascending=False).head(10)

,Country,Period,ALL,NON_SPE,SPE,SPE_SHARE,ALL_CHANGE,SPE_CHANGE,ALL/GDP,SPE/GDP,MLI,GDP
342,Spain,2021,4773.00,4636.00,137.00,2.87,-70.84,109.45,0.39,0.01,0,1209644.65
258,Ireland,2021,125556.00,47441.00,78115.00,62.22,7.62,70.03,29.39,18.28,1,427273.39
27,Indonesia,2021,1210.00,557.00,653.00,53.97,-19.17,32.19,0.12,0.06,1,1005163.55
216,Estonia,2021,331.00,39.00,292.00,88.22,27.80,25.86,1.05,0.93,1,31517.94
195,Cyprus,2021,26620.00,1811.00,24809.00,93.20,23.37,23.56,110.57,103.05,1,24074.46
6,China,2021,21908.00,692.00,21216.00,96.84,8.42,22.28,0.15,0.14,0,15028866.65
265,Italy,2021,54777.00,52288.00,2489.00,4.54,8.74,21.89,3.07,0.14,0,1786188.85
48,Malaysia,2021,15831.00,1184.00,14647.00,92.52,14.66,21.49,5.01,4.63,1,316085.56
251,Hungary,2021,16227.00,14777.00,1450.00,8.94,14.08,20.53,10.53,0.94,1,154108.49
314,Portugal,2021,9973.00,216.00,9757.00,97.83,22.26,16.64,4.64,4.54,1,214968.77


*Countries with the largest share of SPE investments*





*2020*

In [ ]:
dnbin_2020.sort_values(by='SPE_SHARE', ascending=False).head(5)

,Country,Period,ALL,NON_SPE,SPE,SPE_SHARE,ALL_CHANGE,SPE_CHANGE,ALL/GDP,SPE/GDP,MLI,GDP
397,Mauritius,2020,3858.00,-2.00,3860.00,99.95,-10.74,-4.34,38.58,38.60,1,10000.92
124,Ecuador,2020,613.00,-3.00,616.00,99.52,-9.99,-10.85,0.70,0.71,0,87097.48
117,Chile,2020,4372.00,24.00,4348.00,99.45,-15.97,-17.09,1.97,1.96,0,221690.52
446,Liechtenstein,2020,1326.00,31.00,1295.00,97.66,-9.98,-10.94,24.72,24.15,1,5363.11
313,Portugal,2020,8157.00,-208.00,8365.00,97.57,-20.30,-11.82,4.06,4.16,1,200905.14


*2021*

In [ ]:
dnbin_2021.sort_values(by='SPE_SHARE', ascending=False).head(5)

,Country,Period,ALL,NON_SPE,SPE,SPE_SHARE,ALL_CHANGE,SPE_CHANGE,ALL/GDP,SPE/GDP,MLI,GDP
398,Mauritius,2021,3279.00,5.00,3274.00,99.85,-15.01,-15.18,33.56,33.51,1,9770.38
447,Liechtenstein,2021,1292.00,17.00,1275.00,98.68,-2.56,-1.54,NaN,NaN,1,NaN
314,Portugal,2021,9973.00,216.00,9757.00,97.83,22.26,16.64,4.64,4.54,1,214968.77
6,China,2021,21908.00,692.00,21216.00,96.84,8.42,22.28,0.15,0.14,0,15028866.65
279,Lithuania,2021,911.00,31.00,880.00,96.60,1.67,1.38,1.62,1.56,1,56309.54


*Countries with the largest values of SPE investment relative to GDP*

*2020*

In [ ]:
dnbin_2020.sort_values(by='SPE/GDP', ascending=False).head(10)

,Country,Period,ALL,NON_SPE,SPE,SPE_SHARE,ALL_CHANGE,SPE_CHANGE,ALL/GDP,SPE/GDP,MLI,GDP
285,Luxembourg,2020,394116.00,255039.00,139077.00,35.29,-18.41,-44.88,607.21,214.28,1,64905.78
194,Cyprus,2020,21578.00,1500.00,20078.00,93.05,-3.06,-4.98,98.36,91.52,1,21937.24
439,Iceland,2020,19391.00,2012.00,17379.00,89.62,22.48,18.89,101.89,91.32,1,19030.42
397,Mauritius,2020,3858.00,-2.00,3860.00,99.95,-10.74,-4.34,38.58,38.60,1,10000.92
292,Malta,2020,7225.00,2284.00,4941.00,68.39,33.70,-8.16,55.16,37.72,1,13099.18
75,Singapore,2020,102252.00,13977.00,88275.00,86.33,48.29,32.63,33.76,29.14,1,302891.17
446,Liechtenstein,2020,1326.00,31.00,1295.00,97.66,-9.98,-10.94,24.72,24.15,1,5363.11
257,Ireland,2020,116668.00,70725.00,45943.00,39.38,-34.57,-62.06,31.23,12.30,1,373554.63
488,United Kingdom,2020,368727.00,185186.00,183541.00,49.78,-7.40,-30.17,15.54,7.74,1,2372464.18
166,Austria,2020,32911.00,3796.00,29115.00,88.47,-22.87,-20.17,8.62,7.63,1,381776.52


*2021*

In [ ]:
dnbin_2021.sort_values(by='SPE/GDP', ascending=False).head(10)

,Country,Period,ALL,NON_SPE,SPE,SPE_SHARE,ALL_CHANGE,SPE_CHANGE,ALL/GDP,SPE/GDP,MLI,GDP
286,Luxembourg,2021,420224.00,273543.00,146681.00,34.91,6.62,5.47,579.92,202.42,1,72462.92
195,Cyprus,2021,26620.00,1811.00,24809.00,93.20,23.37,23.56,110.57,103.05,1,24074.46
440,Iceland,2021,14652.00,2127.00,12525.00,85.48,-24.44,-27.93,67.53,57.73,1,21696.97
293,Malta,2021,7617.00,2532.00,5085.00,66.76,5.43,2.91,51.76,34.56,1,14715.29
398,Mauritius,2021,3279.00,5.00,3274.00,99.85,-15.01,-15.18,33.56,33.51,1,9770.38
76,Singapore,2021,93492.00,23500.00,69992.00,74.86,-8.57,-20.71,27.79,20.80,1,336429.58
258,Ireland,2021,125556.00,47441.00,78115.00,62.22,7.62,70.03,29.39,18.28,1,427273.39
167,Austria,2021,33207.00,3011.00,30196.00,90.93,0.90,3.71,8.16,7.42,1,407091.87
13,"Hong Kong, China",2021,58657.00,41248.00,17409.00,29.68,-2.87,-8.18,18.75,5.56,0,312861.36
489,United Kingdom,2021,544299.00,404815.00,139484.00,25.63,47.62,-24.00,20.51,5.26,1,2653709.97


### IMF

*Countries with the largest overall investment relative to GDP*

*2020*

In [ ]:
imfin_2020 = imfin[imfin['Period']==2020]
imfin_2020.sort_values(by='DI/GDP', ascending=False).head(15)

,Country,Period,DI,DI_CHANGE,DI/GDP,DI/WDI,MLI,GDP,WDI
67,Bermuda,2020,136865.00,6.41,1987.25,20.17,0,6887.15,678532.00
172,Curacao,2020,28276.00,5.34,1132.77,79.04,0,2496.17,35773.00
121,Cayman Islands,2020,39922.00,10.55,711.75,3.40,0,5608.99,1175830.00
394,Luxembourg,2020,483621.00,-10.88,653.61,19.41,1,73992.59,2491228.00
382,Liberia,2020,6383.00,18.29,209.97,62.74,0,3039.98,10174.00
415,Marshall Islands,2020,476.00,8.68,196.92,7.00,0,241.72,6801.00
289,Guyana,2020,6548.00,159.53,119.68,92.11,0,5471.26,7109.00
307,Iceland,2020,23795.00,33.78,109.68,260.37,1,21694.67,9139.00
175,Cyprus,2020,26478.00,5.89,105.88,6.26,1,25008.45,423112.00
352,Kiribati,2020,150.00,-3.23,82.91,263.16,0,180.91,57.00


*2021*

In [ ]:
imfin_2021 = imfin[imfin['Period']==2021]
imfin_2021.sort_values(by='DI/GDP', ascending=False).head(15)

,Country,Period,DI,DI_CHANGE,DI/GDP,DI/WDI,MLI,GDP,WDI
68,Bermuda,2021,95858.00,-29.96,1315.54,17.20,0,7286.61,557249.00
173,Curacao,2021,28972.00,2.46,1073.19,132.12,0,2699.61,21928.00
395,Luxembourg,2021,475946.00,-1.59,556.62,18.74,1,85506.24,2539578.00
122,Cayman Islands,2021,25072.00,-37.20,425.06,2.72,0,5898.45,921027.00
416,Marshall Islands,2021,343.00,-27.94,132.16,5.54,0,259.54,6196.00
176,Cyprus,2021,30150.00,13.87,106.13,8.32,1,28407.87,362352.00
353,Kiribati,2021,138.00,-8.00,66.66,88.46,0,207.03,156.00
308,Iceland,2021,16595.00,-30.26,64.82,190.66,1,25602.42,8704.00
44,"Bahamas, The",2021,6355.00,265.58,56.70,4.73,0,11208.60,134262.00
413,Malta,2021,8627.00,-2.70,49.68,8.51,1,17364.04,101374.00


*Countries with the largest share of investment into the Netherlands*



*2020*

In [ ]:
imfin_2020.sort_values(by='DI/WDI', ascending=False).head(15)

,Country,Period,DI,DI_CHANGE,DI/GDP,DI/WDI,MLI,GDP,WDI
91,British Indian Ocean Territory,2020,2292.00,-3.29,NaN,4407.69,0,NaN,52.00
352,Kiribati,2020,150.00,-3.23,82.91,263.16,0,180.91,57.00
307,Iceland,2020,23795.00,33.78,109.68,260.37,1,21694.67,9139.00
31,Aruba,2020,1476.00,7.58,56.55,223.64,0,2610.04,660.00
604,South Africa,2020,159593.00,36.82,47.27,170.22,0,337619.68,93757.00
289,Guyana,2020,6548.00,159.53,119.68,92.11,0,5471.26,7109.00
364,Kuwait,2020,4119.00,308.23,3.89,82.33,0,105960.23,5003.00
172,Curacao,2020,28276.00,5.34,1132.77,79.04,0,2496.17,35773.00
571,Saudi Arabia,2020,35488.00,1.04,5.05,66.73,0,703367.84,53179.00
382,Liberia,2020,6383.00,18.29,209.97,62.74,0,3039.98,10174.00


*2021*

In [ ]:
imfin_2021.sort_values(by='DI/WDI', ascending=False).head(15)

,Country,Period,DI,DI_CHANGE,DI/GDP,DI/WDI,MLI,GDP,WDI
92,British Indian Ocean Territory,2021,2125.00,-7.29,NaN,8500.00,0,NaN,25.00
308,Iceland,2021,16595.00,-30.26,64.82,190.66,1,25602.42,8704.00
173,Curacao,2021,28972.00,2.46,1073.19,132.12,0,2699.61,21928.00
605,South Africa,2021,121241.00,-24.03,28.93,120.74,0,419015.02,100415.00
353,Kiribati,2021,138.00,-8.00,66.66,88.46,0,207.03,156.00
572,Saudi Arabia,2021,40473.00,14.05,4.86,87.44,0,833541.24,46288.00
365,Kuwait,2021,3824.00,-7.16,NaN,77.00,0,NaN,4966.00
98,Brunei Darussalam,2021,344.00,-3.91,2.46,40.09,0,14006.57,858.00
599,Solomon Islands,2021,15.00,50.00,0.92,39.47,0,1631.49,38.00
545,Qatar,2021,5853.00,-15.94,3.26,38.18,0,179677.21,15332.00


## Outward Foreign Direct Investment

### DNB

*Context*

In [ ]:
dnbout[(dnbout['Country'] == 'Total')]

,Country,Period,ALL,NON_SPE,SPE,SPE_SHARE,ALL_CHANGE,SPE_CHANGE,ALL/GDP,SPE/GDP,MLI,GDP
490,Total,2015,4543136.00,1761809.00,2781327.00,61.22,NaN,NaN,6.71,4.11,0,67736794.45
491,Total,2016,4952385.00,2025558.00,2926827.00,59.10,9.01,5.23,7.19,4.25,0,68884276.35
492,Total,2017,5068540.00,1990322.00,3078218.00,60.73,2.35,5.17,7.04,4.27,0,72035991.36
493,Total,2018,4985424.00,1913632.00,3071792.00,61.62,-1.64,-0.21,6.80,4.19,0,73268549.37
494,Total,2019,5135148.00,2013941.00,3121207.00,60.78,3.00,1.61,6.56,3.99,0,78254692.53
495,Total,2020,4731164.00,2911108.00,1820056.00,38.47,-7.87,-41.69,6.34,2.44,0,74654038.25
496,Total,2021,4810890.00,3065991.00,1744899.00,36.27,1.69,-4.13,5.88,2.13,0,81790743.53


*Destinations with the largest **increase** in SPE investment from the Netherlands*

*2020*

In [ ]:
dnbout_2020 = dnbout[dnbout['Period']==2020]
dnbout_2020.sort_values(by='SPE_CHANGE', ascending=False).head(5)

,Country,Period,ALL,NON_SPE,SPE,SPE_SHARE,ALL_CHANGE,SPE_CHANGE,ALL/GDP,SPE/GDP,MLI,GDP
124,Ecuador,2020,1172.00,856.00,316.00,26.96,248.81,236.17,1.35,0.36,0,87097.48
194,Cyprus,2020,13586.00,6820.00,6766.00,49.80,7.79,23.53,61.93,30.84,1,21937.24
54,Oman,2020,3648.00,3442.00,206.00,5.65,-58.74,0.98,5.48,0.31,1,66587.19
75,Singapore,2020,65527.00,37098.00,28429.00,43.39,34.31,0.42,21.63,9.39,1,302891.17
187,Croatia,2020,4325.00,1709.00,2616.00,60.49,34.32,-3.47,8.58,5.19,0,50414.05


*2021*

In [ ]:
dnbout_2021 = dnbout[dnbout['Period']==2021]
dnbout_2021.sort_values(by='SPE_CHANGE', ascending=False).head(5)

,Country,Period,ALL,NON_SPE,SPE,SPE_SHARE,ALL_CHANGE,SPE_CHANGE,ALL/GDP,SPE/GDP,MLI,GDP
195,Cyprus,2021,14381.00,5389.00,8992.00,62.53,5.85,32.90,59.74,37.35,1,24074.46
55,Oman,2021,3238.00,2995.00,243.00,7.50,-11.24,17.96,4.33,0.33,1,74738.96
48,Malaysia,2021,13447.00,10542.00,2905.00,21.60,-4.24,16.57,4.25,0.92,1,316085.56
454,Norway,2021,13564.00,7604.00,5960.00,43.94,-1.09,15.55,3.32,1.46,1,408622.76
209,Denmark,2021,27207.00,15655.00,11552.00,42.46,27.14,14.89,8.06,3.42,1,337545.15


*Destinations with the largest share of SPE investments*

*2020*

In [ ]:
dnbout_2020.sort_values(by='SPE_SHARE', ascending=False).head(5)

,Country,Period,ALL,NON_SPE,SPE,SPE_SHARE,ALL_CHANGE,SPE_CHANGE,ALL/GDP,SPE/GDP,MLI,GDP
278,Lithuania,2020,1588.00,225.00,1363.00,85.83,-12.36,-15.18,3.18,2.73,1,49865.46
327,Slovakia,2020,8118.00,1437.00,6681.00,82.30,-0.70,-4.63,8.67,7.14,1,93593.71
145,Panama,2020,4873.00,889.00,3984.00,81.76,-10.96,-15.25,10.29,8.41,0,47348.25
117,Chile,2020,13020.00,3105.00,9915.00,76.15,-5.81,-13.22,5.87,4.47,0,221690.52
313,Portugal,2020,20464.00,5831.00,14633.00,71.51,-13.45,-15.96,10.19,7.28,1,200905.14


*2021*

In [ ]:
dnbout_2021.sort_values(by='SPE_SHARE', ascending=False).head(5)

,Country,Period,ALL,NON_SPE,SPE,SPE_SHARE,ALL_CHANGE,SPE_CHANGE,ALL/GDP,SPE/GDP,MLI,GDP
440,Iceland,2021,203.00,24.00,179.00,88.18,-74.34,-2.19,0.94,0.83,1,21696.97
146,Panama,2021,4690.00,736.00,3954.00,84.31,-3.76,-0.75,8.70,7.34,1,53902.63
279,Lithuania,2021,1494.00,296.00,1198.00,80.19,-5.92,-12.11,2.65,2.13,1,56309.54
118,Chile,2021,12918.00,2564.00,10354.00,80.15,-0.78,4.43,4.81,3.85,1,268693.65
328,Slovakia,2021,9015.00,1936.00,7079.00,78.52,11.05,5.96,9.13,7.17,1,98751.78


*Destinations with the largest values of SPE investment relative to GDP*

*2020*

In [ ]:
dnbout_2020.sort_values(by='SPE/GDP', ascending=False).head(5)

,Country,Period,ALL,NON_SPE,SPE,SPE_SHARE,ALL_CHANGE,SPE_CHANGE,ALL/GDP,SPE/GDP,MLI,GDP
285,Luxembourg,2020,238898.00,150131.00,88767.00,37.16,-16.56,-57.43,368.07,136.76,1,64905.78
257,Ireland,2020,206664.00,72338.00,134326.00,65.00,-17.76,-27.58,55.32,35.96,1,373554.63
194,Cyprus,2020,13586.00,6820.00,6766.00,49.80,7.79,23.53,61.93,30.84,1,21937.24
397,Mauritius,2020,9582.00,6723.00,2859.00,29.84,1.35,-65.85,95.81,28.59,1,10000.92
474,Switzerland,2020,390353.00,307449.00,82904.00,21.24,0.84,-65.07,60.14,12.77,1,649047.03


*2021*

In [ ]:
dnbout_2021.sort_values(by='SPE/GDP', ascending=False).head(5)

,Country,Period,ALL,NON_SPE,SPE,SPE_SHARE,ALL_CHANGE,SPE_CHANGE,ALL/GDP,SPE/GDP,MLI,GDP
286,Luxembourg,2021,246752.00,150082.00,96670.00,39.18,3.29,8.90,340.52,133.41,1,72462.92
195,Cyprus,2021,14381.00,5389.00,8992.00,62.53,5.85,32.90,59.74,37.35,1,24074.46
398,Mauritius,2021,9321.00,6502.00,2819.00,30.24,-2.72,-1.40,95.40,28.85,1,9770.38
258,Ireland,2021,181525.00,86698.00,94827.00,52.24,-12.16,-29.41,42.48,22.19,1,427273.39
475,Switzerland,2021,400219.00,312190.00,88029.00,22.00,2.53,6.18,58.99,12.97,1,678508.61


### IMF

*Destinations with the largest overall investment relative to GDP*

*2020*

In [ ]:
imfout_2020 = imfout[imfout['Period']==2020]
imfout_2020.sort_values(by='DI/GDP', ascending=False).head(20)

,Country,Period,DI,DI_CHANGE,DI/GDP,DI/WDI,MLI,GDP,WDI
67,Bermuda,2020,85480.00,-21.48,1241.15,12.60,0,6887.15,678532.00
172,Curacao,2020,24770.00,-2.79,992.32,69.24,0,2496.17,35773.00
394,Luxembourg,2020,293152.00,-8.86,396.19,11.77,1,73992.59,2491228.00
121,Cayman Islands,2020,21433.00,-14.57,382.12,1.82,0,5608.99,1175830.00
415,Marshall Islands,2020,612.00,50.37,253.18,9.00,0,241.72,6801.00
289,Guyana,2020,6245.00,255.84,114.14,87.85,0,5471.26,7109.00
424,Mauritius,2020,11758.00,10.71,103.13,17.30,1,11401.05,67955.00
541,Puerto Rico,2020,104796.00,10.19,101.72,99.95,0,103020.30,104845.00
175,Cyprus,2020,16671.00,17.74,66.66,3.94,1,25008.45,423112.00
442,Mongolia,2020,8665.00,17.41,65.09,71.61,0,13312.98,12100.00


*2021*

In [ ]:
imfout_2021 = imfout[imfout['Period']==2021]
imfout_2021.sort_values(by='DI/GDP', ascending=False).head(15)

,Country,Period,DI,DI_CHANGE,DI/GDP,DI/WDI,MLI,GDP,WDI
68,Bermuda,2021,77898.00,-8.87,1069.06,13.98,0,7286.61,557249.00
173,Curacao,2021,20447.00,-17.45,757.41,93.25,0,2699.61,21928.00
122,Cayman Islands,2021,21861.00,2.00,370.62,2.37,0,5898.45,921027.00
395,Luxembourg,2021,279471.00,-4.67,326.84,11.00,1,85506.24,2539578.00
416,Marshall Islands,2021,347.00,-43.30,133.70,5.60,0,259.54,6196.00
413,Malta,2021,16964.00,458.58,97.70,16.73,1,17364.04,101374.00
425,Mauritius,2021,10557.00,-10.21,91.57,17.00,1,11529.04,62116.00
542,Puerto Rico,2021,84883.00,-19.00,79.68,98.97,0,106525.70,85767.00
443,Mongolia,2021,9009.00,3.97,58.93,80.28,0,15286.44,11222.00
176,Cyprus,2021,16288.00,-2.30,57.34,4.50,1,28407.87,362352.00


*Destinations with the largest share of investment from the Netherlands*

*2020*

In [ ]:
imfout_2020.sort_values(by='DI/WDI', ascending=False).head(15)

,Country,Period,DI,DI_CHANGE,DI/GDP,DI/WDI,MLI,GDP,WDI
421,Mauritania,2020,3307.00,-1.22,39.34,113.64,0,8405.49,2910.00
541,Puerto Rico,2020,104796.00,10.19,101.72,99.95,0,103020.30,104845.00
160,Cook Islands,2020,88.00,25.71,NaN,96.70,0,NaN,91.00
664,Tokelau,2020,140.00,inf,NaN,93.96,0,NaN,149.00
289,Guyana,2020,6245.00,255.84,114.14,87.85,0,5471.26,7109.00
106,Burundi,2020,253.00,39.01,9.55,84.33,0,2649.67,300.00
601,Somalia,2020,16.00,-5.88,0.23,84.21,0,6883.00,19.00
262,Greenland,2020,163.00,-12.37,5.30,81.50,0,3076.02,200.00
418,Martinique,2020,4.00,33.33,NaN,80.00,0,NaN,5.00
442,Mongolia,2020,8665.00,17.41,65.09,71.61,0,13312.98,12100.00


*2021*

In [ ]:
imfout_2021.sort_values(by='DI/WDI', ascending=False).head(15)

,Country,Period,DI,DI_CHANGE,DI/GDP,DI/WDI,MLI,GDP,WDI
2,Afghanistan,2021,-1740.00,-17500.00,-11.77,140.21,0,14786.86,-1241.00
422,Mauritania,2021,2591.00,-21.65,25.92,108.46,0,9996.25,2389.00
542,Puerto Rico,2021,84883.00,-19.00,79.68,98.97,0,106525.70,85767.00
602,Somalia,2021,151.00,843.75,1.98,98.69,0,7628.00,153.00
161,Cook Islands,2021,91.00,3.41,NaN,97.85,0,NaN,93.00
665,Tokelau,2021,128.00,-8.57,NaN,96.24,0,NaN,133.00
173,Curacao,2021,20447.00,-17.45,757.41,93.25,0,2699.61,21928.00
443,Mongolia,2021,9009.00,3.97,58.93,80.28,0,15286.44,11222.00
419,Martinique,2021,8.00,100.00,NaN,80.00,0,NaN,10.00
107,Burundi,2021,222.00,-12.25,7.99,79.86,0,2779.81,278.00
